In [ ]:
import pandas as pd

# โหลดและคลีนข้อมูลสภาพอากาศ
file_path = 'rain.xlsx'
xls = pd.ExcelFile(file_path)

df_list = []
for sheet in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet)
    df = df.rename(columns={'ปี': 'year', 'เดือน': 'month', 'วันที่': 'day'})
    df['year'] = pd.to_numeric(df['year'], errors='coerce')
    df['month'] = pd.to_numeric(df['month'], errors='coerce')
    df['day'] = pd.to_numeric(df['day'], errors='coerce')
    df = df.dropna(subset=['year', 'month', 'day'])
    df['วัน'] = pd.to_datetime(df[['year', 'month', 'day']], errors='coerce')
    df = df.dropna(subset=['วัน'])
    for col in ['อุณหภูมิ', 'ฝน', 'ชื้น']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    df = df[df[['อุณหภูมิ', 'ฝน', 'ชื้น']].notna().all(axis=1)]
    df_list.append(df)

weather = pd.concat(df_list, ignore_index=True)
weather = weather.drop_duplicates()
weather = weather.sort_values('วัน').reset_index(drop=True)

# โหลดและคลีนข้อมูลผู้ป่วย
xls = pd.ExcelFile('DHFcc12.xlsx')
df_list = []
for sheet in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet)
    df.columns = df.columns.str.strip()
    if 'วันเริ่มป่วย' in df.columns:
        df['วันเริ่มป่วย'] = pd.to_datetime(df['วันเริ่มป่วย'], errors='coerce')
    for col in ['อายุ(ปี)', 'อายุ(เดือน)']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    df_list.append(df)

patients = pd.concat(df_list, ignore_index=True)
patients = patients.drop_duplicates()
patients = patients.dropna(subset=['วันเริ่มป่วย'])

# สร้างคอลัมน์ ปี เดือน ไตรมาส ฤดูกาล จากวันเริ่มป่วย
patients['ปี'] = patients['วันเริ่มป่วย'].dt.year
patients['เดือน'] = patients['วันเริ่มป่วย'].dt.month
patients['ไตรมาส'] = patients['วันเริ่มป่วย'].dt.quarter

def assign_season(row):
    month = row['เดือน']
    day = row['วันเริ่มป่วย'].day
    if (month == 2 and day >= 15) or month in [3, 4] or (month == 5 and day < 15):
        return 'Summer'
    elif (month == 5 and day >= 15) or month in [6, 7, 8, 9] or (month == 10 and day < 15):
        return 'Rainy'
    else:
        return 'Winter'

patients['ฤดูกาล'] = patients.apply(assign_season, axis=1)

# เลือกคอลัมน์ที่ต้องการใช้วิเคราะห์
selected_cols = ['วันเริ่มป่วย', 'เพศ', 'อายุ(ปี)', 'อายุ(เดือน)', 'ตำบล', 'อำเภอ',
                 'เดือน', 'ปี', 'ไตรมาส', 'ฤดูกาล', 'อาชีพ']

df_selected = patients[selected_cols]

# นับจำนวนผู้ป่วยรายวัน
daily_counts = df_selected.groupby('วันเริ่มป่วย').size().reset_index(name='จำนวนผู้ป่วย')

# หา mode หรือ mean ของแต่ละตัวแปร (รายวัน)
daily_modes = df_selected.groupby('วันเริ่มป่วย').agg({
    'เพศ': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'อายุ(ปี)': 'mean',
    'อายุ(เดือน)': 'mean',
    'ตำบล': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'อำเภอ': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'เดือน': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'ปี': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'ไตรมาส': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'ฤดูกาล': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'อาชีพ': lambda x: x.mode().iloc[0] if not x.mode().empty else None
}).reset_index()

# รวมจำนวนผู้ป่วยกับข้อมูลตัวแปรรายวัน
df_analysis = pd.merge(daily_counts, daily_modes, on='วันเริ่มป่วย', how='left')

# รวมกับข้อมูลสภาพอากาศ (merge โดยใช้วันเริ่มป่วย = วัน)
df_analysis = pd.merge(
    df_analysis,
    weather[['วัน', 'อุณหภูมิ', 'ฝน', 'ชื้น']],
    left_on='วันเริ่มป่วย',
    right_on='วัน',
    how='left'
).drop(columns=['วัน'])
# เรียงข้อมูลตามวันก่อน เพื่อ rolling ให้ถูกต้อง
df_analysis = df_analysis.sort_values('วันเริ่มป่วย')

# สร้างค่าเฉลี่ยย้อนหลัง 15 วัน
df_analysis['temp_avg15'] = df_analysis['อุณหภูมิ'].rolling(window=15).mean()
df_analysis['rain_avg15'] = df_analysis['ฝน'].rolling(window=15).mean()
df_analysis['humidity_avg15'] = df_analysis['ชื้น'].rolling(window=15).mean()

# ลบข้อมูลแถวที่ยังไม่ครบ 15 วันแรก (NaN)
df_analysis = df_analysis.dropna(subset=['temp_avg15', 'rain_avg15', 'humidity_avg15'])


# แสดงข้อมูลตัวอย่าง
print(df_analysis.head())


In [ ]:
print(df_analysis.columns)


In [ ]:
df_analysis = df_analysis.rename(columns={
    'วันเริ่มป่วย': 'date',
    'จำนวนผู้ป่วย': 'cases',
    'เพศ': 'gender',
    'อายุ(ปี)': 'age_year',
    'อายุ(เดือน)': 'age_month',
    'ตำบล': 'subdistrict',
    'อำเภอ': 'district',
    'เดือน': 'month',
    'ปี': 'year',
    'ไตรมาส': 'quarter',
    'ฤดูกาล': 'season',
    'อาชีพ': 'occupation',
    'อุณหภูมิ': 'temperature',
    'ฝน': 'rainfall',
    'ชื้น': 'humidity'
})


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

numeric_cols = ['cases', 'temperature', 'rainfall', 'humidity',
                'temp_avg15', 'rain_avg15', 'humidity_avg15',
                'age_year', 'age_month', 'month', 'year', 'quarter']

df_corr = df_analysis[numeric_cols].copy()

# คำนวณค่า Correlation matrix
correlation_matrix = df_corr.corr()

# วาด heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", square=True)
plt.title('Correlation Matrix with Dengue Cases')
plt.tight_layout()
plt.show()

# แสดงคอลัมน์ที่สัมพันธ์กับจำนวนผู้ป่วย เรียงจากมากไปน้อย
print(correlation_matrix['cases'].sort_values(ascending=False))


In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

# สมมติ df_analysis มีข้อมูลครบและมีคอลัมน์:
# 'จำนวนผู้ป่วย' (cases), 'year', 'rain_avg15', 'temp_avg15', 'เพศ', 'อำเภอ', 'ฤดูกาล'

# เปลี่ยนชื่อคอลัมน์ภาษาไทยเป็นอังกฤษให้สะดวก
df_analysis = df_analysis.rename(columns={
    'จำนวนผู้ป่วย': 'cases',
    'เพศ': 'gender',
    'อำเภอ': 'district',
    'ฤดูกาล': 'season'
})

# สร้าง dummy variables สำหรับ categorical ตัวแปร gender, district, season
df_analysis = pd.get_dummies(df_analysis, columns=['gender', 'district', 'season'], drop_first=True)

# ดู columns ที่ใช้
print(df_analysis.columns)


In [ ]:
print(df_analysis.columns)


In [ ]:
# เปลี่ยนชื่อคอลัมน์เป็นอังกฤษง่ายๆ
df_analysis = df_analysis.rename(columns={
    'วันเริ่มป่วย': 'date',
    'จำนวนผู้ป่วย': 'cases',
    'เพศ': 'gender',
    'อายุ(ปี)': 'age_year',
    'อายุ(เดือน)': 'age_month',
    'ตำบล': 'district',
    'อำเภอ': 'subdistrict',
    'เดือน': 'month',
    'ปี': 'year',
    'ไตรมาส': 'quarter',
    'ฤดูกาล': 'season',
    'อาชีพ': 'occupation',
    'อุณหภูมิ': 'temperature',
    'ฝน': 'rainfall',
    'ชื้น': 'humidity',
    'temp_avg15': 'temp_avg15',
    'rain_avg15': 'rain_avg15',
    'humidity_avg15': 'humidity_avg15'
})

# สร้าง dummy variables สำหรับ categorical variables
df_analysis = pd.get_dummies(df_analysis, columns=['gender', 'district', 'season'], drop_first=True)

# ตรวจสอบคอลัมน์หลังสร้าง dummy
print(df_analysis.columns)

# สร้างสูตรโมเดล (ใช้ตัวแปรที่ต้องการ)
numeric_vars = ['year', 'rain_avg15', 'temp_avg15', 'humidity_avg15', 'age_year', 'age_month', 'month', 'quarter']
dummy_vars = [col for col in df_analysis.columns if col.startswith('gender_') or col.startswith('district_') or col.startswith('season_')]
all_vars = numeric_vars + dummy_vars

formula = 'cases ~ ' + ' + '.join(all_vars)
print(formula)

# Fit โมเดล Negative Binomial
nb_model = smf.glm(formula=formula, data=df_analysis, family=sm.families.NegativeBinomial()).fit()

# ดูสรุปผลโมเดล
print(nb_model.summary())


In [ ]:
df_analysis.to_excel('summary.xlsx', index=False)
print("บันทึกไฟล์ summary.xlsx เรียบร้อยแล้ว")


In [ ]:
print("จำนวนวันที่ผู้ป่วยเริ่มป่วยที่ไม่ซ้ำ:", patients['วันเริ่มป่วย'].nunique())
print("จำนวนแถวรวมข้อมูลผู้ป่วย:", len(patients))


In [ ]:
# เลือกเฉพาะตัวเลข
num_cols = ['จำนวนผู้ป่วย', 'อายุ(ปี)', 'อายุ(เดือน)', 'เดือน', 'ปี', 'อุณหภูมิ', 'ฝน', 'ชื้น']
print(df_analysis[num_cols].corr()['จำนวนผู้ป่วย'])


In [ ]:
import pandas as pd

# สมมติ df_analysis โหลดมาแล้ว พร้อมข้อมูลครบถ้วน

# 1. ดู correlation ของตัวแปรเชิงตัวเลขกับจำนวนผู้ป่วย
num_cols = ['จำนวนผู้ป่วย', 'อายุ(ปี)', 'อายุ(เดือน)', 'เดือน', 'ปี', 'อุณหภูมิ', 'ฝน', 'ชื้น']
print("Correlation matrix กับ จำนวนผู้ป่วย:")
print(df_analysis[num_cols].corr()['จำนวนผู้ป่วย'].sort_values(ascending=False))
print("\n")

# 2. ดูค่าเฉลี่ยจำนวนผู้ป่วยตามกลุ่มของตัวแปรเชิงหมวดหมู่
categorical_cols = ['เพศ', 'ตำบล', 'อำเภอ', 'ไตรมาส', 'ฤดูกาล', 'อาชีพ']

for col in categorical_cols:
    print(f"ค่าเฉลี่ยจำนวนผู้ป่วยตาม {col}:")
    mean_cases = df_analysis.groupby(col)['จำนวนผู้ป่วย'].mean().sort_values(ascending=False)
    print(mean_cases)
    print("\n")

# 3. (ถ้าต้องการ) สรุปข้อมูลใน DataFrame เพื่อง่ายต่อการวิเคราะห์
summary = {}

# correlation
summary['correlation'] = df_analysis[num_cols].corr()['จำนวนผู้ป่วย']

# ค่าเฉลี่ยจำนวนผู้ป่วยตามกลุ่ม categorical
for col in categorical_cols:
    summary[col] = df_analysis.groupby(col)['จำนวนผู้ป่วย'].mean()

# แสดงสรุปแบบง่าย
print("สรุปความสัมพันธ์:")
for key, value in summary.items():
    print(f"--- {key} ---")
    print(value)
    print()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
sns.heatmap(df_analysis[num_cols].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix of Numeric Variables')
plt.show()


In [ ]:
# จำนวนวันที่มีผู้ป่วย = 0
num_zero_days = (df_analysis['จำนวนผู้ป่วย'] == 0).sum()

# จำนวนวันทั้งหมด
total_days = df_analysis.shape[0]

# เปอร์เซ็นต์วันที่มีผู้ป่วย = 0
percent_zero = num_zero_days / total_days * 100

print(f"จำนวนวันที่มีผู้ป่วย = 0: {num_zero_days} วัน")
print(f"จำนวนวันทั้งหมด: {total_days} วัน")
print(f"เปอร์เซ็นต์วันที่มีผู้ป่วย = 0: {percent_zero:.2f}%")


In [ ]:
# เปลี่ยนชื่อคอลัมน์ใน df_analysis เป็นอังกฤษเพื่อความง่ายในการใช้สูตรโมเดล
df_analysis = df_analysis.rename(columns={
    'วันเริ่มป่วย': 'date',
    'จำนวนผู้ป่วย': 'cases',
    'เพศ': 'gender',
    'อายุ(ปี)': 'age_year',
    'อายุ(เดือน)': 'age_month',
    'ตำบล': 'subdistrict',
    'อำเภอ': 'district',
    'เดือน': 'month',
    'ปี': 'year',
    'ไตรมาส': 'quarter',
    'ฤดูกาล': 'season',
    'อาชีพ': 'occupation',
    'อุณหภูมิ': 'temperature',
    'ฝน': 'rainfall',
    'ชื้น': 'humidity'
})

# ดูตัวอย่างข้อมูลที่แปลงแล้ว
print(df_analysis.head())


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.histplot(df_analysis['cases'], bins=30)
plt.title('Distribution of Daily Dengue Cases')
plt.show()
print(df_analysis['cases'].describe())


In [ ]:
mean_cases = df_analysis['cases'].mean()
var_cases = df_analysis['cases'].var()
print(f"Mean: {mean_cases}, Variance: {var_cases}")

if var_cases > mean_cases:
    print("พบ Overdispersion แนะนำใช้ Negative Binomial Regression")
else:
    print("Poisson Regression น่าจะเหมาะสม")


In [ ]:
import statsmodels.formula.api as smf

# สูตรโมเดล รวมตัวแปร numeric กับ categorical (ใช้ C() ครอบ categorical variables)
formula = (
    'cases ~ temperature + rainfall + humidity + month + year + age_year + age_month + '
    'C(gender) + C(subdistrict) + C(district) + C(quarter) + C(season) + C(occupation)'
)

# Fit Negative Binomial Regression
nb_model = smf.glm(formula=formula, data=df_analysis, family=sm.families.NegativeBinomial()).fit()

# แสดงสรุปผลโมเดล
print(nb_model.summary())


In [ ]:
# ดึงผลสรุปตารางแบบ DataFrame
summary_df = nb_model.summary2().tables[1]

# กรองเฉพาะตัวแปรที่ p-value < 0.05 (นัยสำคัญ)
significant_vars = summary_df[summary_df['P>|z|'] < 0.05]

print("ตัวแปรที่มีนัยสำคัญ (p < 0.05):")
print(significant_vars)


In [ ]:
sig_var_names = significant_vars.index.tolist()
print("ชื่อตัวแปรที่มีนัยสำคัญ:", sig_var_names)


In [ ]:
# ดึงตัวแปรที่มีนัยสำคัญ (p < 0.05)
summary_df = nb_model.summary2().tables[1]
significant_vars = summary_df[summary_df['P>|z|'] < 0.05]
sig_var_names = significant_vars.index.tolist()

# ลบ intercept ออก เพราะไม่ต้องใส่ในสูตร
if 'Intercept' in sig_var_names:
    sig_var_names.remove('Intercept')

print("ตัวแปรนัยสำคัญ:", sig_var_names)

# สร้างสูตรใหม่
# ต้องแปลงชื่อ categorical variables ให้เหมาะสม เช่น 'C(gender)[T.male]' ให้เป็น 'C(gender)'
# เอาแค่ชื่อหลักมาใช้ โดยกรองชื่อที่ขึ้นต้นด้วย 'C(' หรือไม่มี 'C(' ก็เอา

def clean_var_name(var):
    # ถ้าเป็น dummy variable เช่น 'C(gender)[T.male]' ให้ return 'C(gender)'
    if var.startswith('C('):
        return var.split('[')[0]
    else:
        return var

clean_vars = list(set([clean_var_name(v) for v in sig_var_names]))

# สร้างสูตรใหม่ (รวมตัวแปร categorical ด้วย C())
formula_new = 'cases ~ ' + ' + '.join(clean_vars)

print("สูตรใหม่:", formula_new)

# Fit โมเดลใหม่
nb_model2 = smf.glm(formula=formula_new, data=df_analysis, family=sm.families.NegativeBinomial()).fit()

# แสดงสรุปโมเดลใหม่
print(nb_model2.summary())


In [ ]:
import statsmodels.formula.api as smf

# ดึงตารางสรุป coefficients
summary_df = nb_model.summary2().tables[1]

# เลือกตัวแปรที่ p-value < 0.05 (นัยสำคัญ)
significant_vars = summary_df[summary_df['P>|z|'] < 0.05]
sig_var_names = significant_vars.index.tolist()

# ลบ intercept ออก
if 'Intercept' in sig_var_names:
    sig_var_names.remove('Intercept')

print("ตัวแปรนัยสำคัญ:", sig_var_names)

# ฟังก์ชันแปลงชื่อตัวแปร dummy ให้เหลือเฉพาะชื่อหลัก
def clean_var_name(var):
    if var.startswith('C('):
        return var.split('[')[0]  # เอาแค่ 'C(variable)'
    else:
        return var

clean_vars = list(set([clean_var_name(v) for v in sig_var_names]))

# สร้างสูตรโมเดลใหม่จากตัวแปรนัยสำคัญ
formula_new = 'cases ~ ' + ' + '.join(clean_vars)
print("สูตรโมเดลใหม่:", formula_new)

# Fit โมเดล Negative Binomial ใหม่ด้วยสูตรนี้
nb_model2 = smf.glm(formula=formula_new, data=df_analysis, family=sm.families.NegativeBinomial()).fit()

# แสดงผลสรุปโมเดลใหม่
print(nb_model2.summary())


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# 1. เลือก features และ target
target = 'cases'
features = df_analysis.drop(columns=['date', 'cases'])  # เอา 'date' กับ target ออก

# แยกตัวแปรตามประเภท
categorical_cols = features.select_dtypes(include='object').columns.tolist()
numeric_cols = features.select_dtypes(exclude='object').columns.tolist()

# 2. One-hot encoding ตัวแปรหมวดหมู่
df_encoded = pd.get_dummies(df_analysis, columns=categorical_cols, drop_first=True)

# 3. แยก X, y
X = df_encoded.drop(columns=['date', 'cases'])
y = df_encoded['cases']

# 4. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. Train Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# 6. Evaluate
y_pred = rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse:.4f}')
print(f'R²: {r2:.4f}')


In [ ]:
from xgboost import XGBRegressor

xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print(f"[XGBoost] MSE: {mse_xgb:.4f}, R²: {r2_xgb:.4f}")


In [ ]:
import pandas as pd

# --- โหลดและคลีนข้อมูลสภาพอากาศ ---
file_path = 'rain.xlsx'
xls = pd.ExcelFile(file_path)

df_list = []
for sheet in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet)
    df = df.rename(columns={'ปี': 'year', 'เดือน': 'month', 'วันที่': 'day'})
    df['year'] = pd.to_numeric(df['year'], errors='coerce')
    df['month'] = pd.to_numeric(df['month'], errors='coerce')
    df['day'] = pd.to_numeric(df['day'], errors='coerce')
    df = df.dropna(subset=['year', 'month', 'day'])
    df['วัน'] = pd.to_datetime(df[['year', 'month', 'day']], errors='coerce')
    df = df.dropna(subset=['วัน'])
    for col in ['อุณหภูมิ', 'ฝน', 'ชื้น']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    df = df[df[['อุณหภูมิ', 'ฝน', 'ชื้น']].notna().all(axis=1)]
    df_list.append(df)

weather = pd.concat(df_list, ignore_index=True)
weather = weather.drop_duplicates()
weather = weather.sort_values('วัน').reset_index(drop=True)

# --- โหลดและคลีนข้อมูลผู้ป่วย ---
xls = pd.ExcelFile('DHFcc12.xlsx')
df_list = []
for sheet in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet)
    df.columns = df.columns.str.strip()
    if 'วันเริ่มป่วย' in df.columns:
        df['วันเริ่มป่วย'] = pd.to_datetime(df['วันเริ่มป่วย'], errors='coerce')
    for col in ['อายุ(ปี)', 'อายุ(เดือน)']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    df_list.append(df)

patients = pd.concat(df_list, ignore_index=True)
patients = patients.drop_duplicates()
patients = patients.dropna(subset=['วันเริ่มป่วย'])

# สร้างคอลัมน์ ปี เดือน ไตรมาส ฤดูกาล จากวันเริ่มป่วย
patients['ปี'] = patients['วันเริ่มป่วย'].dt.year
patients['เดือน'] = patients['วันเริ่มป่วย'].dt.month
patients['ไตรมาส'] = patients['วันเริ่มป่วย'].dt.quarter

def assign_season(row):
    month = row['เดือน']
    day = row['วันเริ่มป่วย'].day
    if (month == 2 and day >= 15) or month in [3, 4] or (month == 5 and day < 15):
        return 'Summer'
    elif (month == 5 and day >= 15) or month in [6, 7, 8, 9] or (month == 10 and day < 15):
        return 'Rainy'
    else:
        return 'Winter'

patients['ฤดูกาล'] = patients.apply(assign_season, axis=1)

# เลือกคอลัมน์ที่ใช้วิเคราะห์ (ตัดอายุ(เดือน)ออกตามคำขอ)
selected_cols = ['วันเริ่มป่วย', 'เพศ', 'อายุ(ปี)', 'ตำบล', 'อำเภอ',
                 'เดือน', 'ปี', 'ไตรมาส', 'ฤดูกาล', 'อาชีพ']

df_selected = patients[selected_cols]

# นับจำนวนผู้ป่วยรายวัน
daily_counts = df_selected.groupby('วันเริ่มป่วย').size().reset_index(name='จำนวนผู้ป่วย')

# หา mode หรือ mean ของแต่ละตัวแปร (รายวัน)
daily_modes = df_selected.groupby('วันเริ่มป่วย').agg({
    'เพศ': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'อายุ(ปี)': 'mean',
    'ตำบล': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'อำเภอ': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'เดือน': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'ปี': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'ไตรมาส': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'ฤดูกาล': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'อาชีพ': lambda x: x.mode().iloc[0] if not x.mode().empty else None
}).reset_index()

# รวมจำนวนผู้ป่วยกับข้อมูลตัวแปรรายวัน
df_analysis = pd.merge(daily_counts, daily_modes, on='วันเริ่มป่วย', how='left')

# รวมกับข้อมูลสภาพอากาศ
df_analysis = pd.merge(
    df_analysis,
    weather[['วัน', 'อุณหภูมิ', 'ฝน', 'ชื้น']],
    left_on='วันเริ่มป่วย',
    right_on='วัน',
    how='left'
).drop(columns=['วัน'])

# เปลี่ยนชื่อคอลัมน์เป็นอังกฤษ
df_analysis = df_analysis.rename(columns={
    'วันเริ่มป่วย': 'date',
    'จำนวนผู้ป่วย': 'cases',
    'เพศ': 'gender',
    'อายุ(ปี)': 'age_year',
    'ตำบล': 'district',
    'อำเภอ': 'subdistrict',
    'เดือน': 'month',
    'ปี': 'year',
    'ไตรมาส': 'quarter',
    'ฤดูกาล': 'season',
    'อาชีพ': 'occupation',
    'อุณหภูมิ': 'temperature',
    'ฝน': 'rainfall',
    'ชื้น': 'humidity'
})

# เรียงตามวันที่
df_analysis = df_analysis.sort_values('date').reset_index(drop=True)

# สร้างฟีเจอร์ rolling mean 15 วัน (ค่าเฉลี่ยย้อนหลัง)
window_size = 15
for col in ['cases', 'temperature', 'rainfall', 'humidity']:
    df_analysis[f'{col}_rolling_mean_{window_size}d'] = df_analysis[col].rolling(window=window_size, min_periods=1).mean()

# ดูข้อมูลตัวอย่าง
print(df_analysis.head(20))


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# 1. เตรียมข้อมูล
# เลือกฟีเจอร์ที่ใช้
features = [
    'age_year', 'month', 'year', 'quarter', 'temperature', 'rainfall', 'humidity',
    'cases_rolling_mean_15d', 'temperature_rolling_mean_15d', 'rainfall_rolling_mean_15d', 'humidity_rolling_mean_15d'
]

# แปลง categorical เป็น dummy variables
df_model = df_analysis.copy()
df_model = pd.get_dummies(df_model, columns=['gender', 'district', 'subdistrict', 'season', 'occupation'], drop_first=True)

# Update features list เพิ่ม dummy columns ที่สร้างใหม่
dummy_cols = [col for col in df_model.columns if col.startswith(('gender_', 'district_', 'subdistrict_', 'season_', 'occupation_'))]
features += dummy_cols

# กรองข้อมูลที่ไม่มีค่า missing
df_model = df_model.dropna(subset=['cases'] + features)

X = df_model[features]
y = df_model['cases']

# 2. แบ่งข้อมูล train-test (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. สร้างโมเดล Random Forest Regression
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 4. ทำนายและประเมินผล
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f'RMSE: {rmse:.2f}')
print(f'R^2: {r2:.2f}')

# 5. ดูความสำคัญของตัวแปร (feature importance)
importances = model.feature_importances_
feature_importance = pd.Series(importances, index=features).sort_values(ascending=False)

print("Feature Importances:")
print(feature_importance.head(10))


In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

# กำหนด model เบื้องต้น
xgb = XGBRegressor(objective='reg:squarederror', random_state=42)

# กำหนดพารามิเตอร์ที่ต้องการลอง
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 1],
    'colsample_bytree': [0.7, 1]
}

# สร้าง GridSearchCV
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid,
                           cv=3, scoring='neg_mean_squared_error', verbose=1)

# Fit model
grid_search.fit(X_train, y_train)

# best params
print("Best parameters:", grid_search.best_params_)

# ทำนายด้วย model ที่ดีที่สุด
best_xgb = grid_search.best_estimator_
y_pred = best_xgb.predict(X_test)

# ประเมินผล
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.2f}")
print(f"R^2: {r2:.2f}")


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

# สมมติ X_train, X_test, y_train, y_test เตรียมไว้แล้ว (Feature เป็น numeric เท่านั้น)

# สเกลข้อมูล (StandardScaler)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# สร้างโมเดลง่าย ๆ ANN
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1)  # output layer สำหรับ regression (ไม่มี activation)
])

model.compile(optimizer='adam', loss='mse')

# ฝึกโมเดล
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32,
                    validation_split=0.2, verbose=1)

# ทำนายและประเมินผล
y_pred = model.predict(X_test_scaled).flatten()

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Deep Learning RMSE: {rmse:.2f}")
print(f"Deep Learning R^2: {r2:.2f}")

# วาดกราฟ loss train กับ validation
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.legend()
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# สมมติ X_train, X_test, y_train, y_test เตรียมไว้แล้ว

# สเกลข้อมูล
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# สร้างโมเดล Deep Learning ใหม่
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.4),  # เพิ่ม Dropout
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),  # เพิ่ม Dropout
    Dense(1)  # output layer สำหรับ regression
])

# ปรับ optimizer และ learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='mse')

# Early stopping เพื่อตัดการเทรนเมื่อ val_loss ไม่ลดลง 10 epochs
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# ฝึกโมเดล
history = model.fit(
    X_train_scaled, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    verbose=1,
    callbacks=[early_stop]
)

import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

y_pred = model.predict(X_test_scaled).flatten()

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)  # แปลง MSE เป็น RMSE
r2 = r2_score(y_test, y_pred)

print(f"Deep Learning RMSE: {rmse:.2f}")
print(f"Deep Learning R^2: {r2:.2f}")



# อันที่สอง

In [ ]:
import pandas as pd

#  โหลดและคลีนข้อมูลสภาพอากาศจากทุกชีท
file_path = 'rain.xlsx'
xls = pd.ExcelFile(file_path)

df_list = []
for sheet in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet)
    df = df.rename(columns={'ปี': 'year', 'เดือน': 'month', 'วันที่': 'day'})  # เปลี่ยนชื่อเป็นอังกฤษ

    #  แปลงวันที่เป็นตัวเลข และกรองค่าที่ว่าง
    df['year'] = pd.to_numeric(df['year'], errors='coerce')
    df['month'] = pd.to_numeric(df['month'], errors='coerce')
    df['day'] = pd.to_numeric(df['day'], errors='coerce')
    df = df.dropna(subset=['year', 'month', 'day'])

    # แปลงเป็น datetime และกรองค่าที่แปลงไม่ได้
    df['วัน'] = pd.to_datetime(df[['year', 'month', 'day']], errors='coerce')
    df = df.dropna(subset=['วัน'])

    #  แปลงคอลัมน์อุณหภูมิ/ฝน/ชื้น เป็นตัวเลข (0 ไม่ถูกลบ)
    for col in ['อุณหภูมิ', 'ฝน', 'ชื้น']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    #  กรองเฉพาะแถวที่ไม่มี NaN ในคอลัมน์หลัก (แต่ 0 ยังอยู่)
    df = df[df[['อุณหภูมิ', 'ฝน', 'ชื้น']].notna().all(axis=1)]

    df_list.append(df)

#  รวมทุกชีทเข้าด้วยกัน
weather = pd.concat(df_list, ignore_index=True)
weather = weather.drop_duplicates()
weather = weather.sort_values('วัน').reset_index(drop=True)

#  บันทึกไฟล์ที่คลีนแล้ว
weather.to_excel('rain_all_years_sorted3.xlsx', index=False)
print(" บันทึกไฟล์ rain_all_years_sorted3.xlsx เรียบร้อยแล้ว")

#  โหลดและคลีนข้อมูลผู้ป่วย
xls = pd.ExcelFile('DHFcc12.xlsx')
df_list = []

for sheet in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet)
    df.columns = df.columns.str.strip()  # ลบช่องว่างจากชื่อคอลัมน์

    #  แปลงวันเริ่มป่วยเป็น datetime
    if 'วันเริ่มป่วย' in df.columns:
        df['วันเริ่มป่วย'] = pd.to_datetime(df['วันเริ่มป่วย'], errors='coerce')

    #  แปลงอายุเป็นตัวเลข (0 ได้)
    for col in ['อายุ(ปี)', 'อายุ(เดือน)']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    df_list.append(df)

#  รวมข้อมูลผู้ป่วย
patients = pd.concat(df_list, ignore_index=True)
patients = patients.drop_duplicates()
patients = patients.dropna(subset=['วันเริ่มป่วย'])

print(f" จำนวนผู้ป่วยที่มีวันเริ่มป่วย: {len(patients)}")

#  ฟังก์ชันคำนวณ rolling average
def get_rolling_avg(date, n_days=15):
    start_date = date - pd.Timedelta(days=n_days)
    mask = (weather['วัน'] >= start_date) & (weather['วัน'] < date)
    subset = weather.loc[mask]
    if subset.empty:
        return pd.Series([None, None, None])
    return pd.Series([
        subset['อุณหภูมิ'].mean(),
        subset['ฝน'].mean(),
        subset['ชื้น'].mean()  #ฟังก์ชันนี้รับวันที่ date แล้วคืนค่าเฉลี่ยอุณหภูมิ ฝน และความชื้น ย้อนหลัง n_days วัน
    ])

#  เพิ่มค่าเฉลี่ยย้อนหลัง 15 วัน
patients[['temp_15d_avg', 'rain_15d_avg', 'humid_15d_avg']] = patients['วันเริ่มป่วย'].apply(get_rolling_avg)

print(" คำนวณค่าเฉลี่ย temp, rain และ humid ย้อนหลัง 15 วันเสร็จสิ้น")

# สร้างคอลัมน์ ปี เดือน ไตรมาส ฤดูกาล
patients['ปี'] = patients['วันเริ่มป่วย'].dt.year
patients['เดือน'] = patients['วันเริ่มป่วย'].dt.month
patients['ไตรมาส'] = patients['วันเริ่มป่วย'].dt.quarter

#  ฟังก์ชันแบ่งฤดูแบบไทย
def assign_season(row):
    month = row['เดือน']
    day = row['วันเริ่มป่วย'].day

    if (month == 2 and day >= 15) or month in [3, 4] or (month == 5 and day < 15):
        return 'Summer'
    elif (month == 5 and day >= 15) or month in [6, 7, 8, 9] or (month == 10 and day < 15):
        return 'Rainy'
    else:
        return 'Winter'

patients['ฤดูกาล'] = patients.apply(assign_season, axis=1)

#  สรุปรายวัน
if 'วันพบผป' in patients.columns:
    daily_counts = patients.groupby('วันพบผป').size().reset_index(name='จำนวนผู้ป่วย')

    daily_weather = patients.groupby('วันพบผป').agg({
        'temp_15d_avg': 'mean',
        'rain_15d_avg': 'mean',
        'humid_15d_avg': 'mean'
    }).reset_index()

    df_analysis = pd.merge(daily_counts, daily_weather, on='วันพบผป', how='left')

    daily_vars = patients.groupby('วันพบผป').agg({
        'อายุ(ปี)': 'mean',
        'เพศ': lambda x: x.mode()[0] if not x.mode().empty else None,
        'อาชีพ': lambda x: x.mode()[0] if not x.mode().empty else None,
        'ตำบล': lambda x: x.mode()[0] if not x.mode().empty else None,
        'อำเภอ': lambda x: x.mode()[0] if not x.mode().empty else None,
        'เดือน': lambda x: x.mode()[0] if not x.mode().empty else None,
        'ปี': lambda x: x.mode()[0] if not x.mode().empty else None,
        'ไตรมาส': lambda x: x.mode()[0] if not x.mode().empty else None,
        'ฤดูกาล': lambda x: x.mode()[0] if not x.mode().empty else None
    }).reset_index()

    df_analysis = pd.merge(df_analysis, daily_vars, on='วันพบผป', how='left')
    print("✅ รวมข้อมูลผู้ป่วยกับสภาพอากาศรายวันเสร็จแล้ว")
else:
    print("⚠️ ไม่พบคอลัมน์ 'วันพบผป' ในข้อมูลผู้ป่วย")


In [ ]:
import pandas as pd

# โหลดไฟล์
xls = pd.ExcelFile('DHFcc12.xlsx')

df_list = []

for sheet in xls.sheet_names:
    if sheet == "Table1":
        continue  # ข้ามชีทที่ไม่ใช่ข้อมูลผู้ป่วย

    df = pd.read_excel(xls, sheet_name=sheet)
    df.columns = df.columns.str.strip()  # ลบช่องว่างจากชื่อคอลัมน์

    # แปลงวันเริ่มป่วยเป็น datetime
    if 'วันเริ่มป่วย' in df.columns:
        df['วันเริ่มป่วย'] = pd.to_datetime(df['วันเริ่มป่วย'], errors='coerce')

    # แปลงอายุให้เป็นตัวเลข (0 ได้)
    for col in ['อายุ(ปี)', 'อายุ(เดือน)']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # ลบแถวที่ไม่มีวันเริ่มป่วย
    df = df.dropna(subset=['วันเริ่มป่วย'])

    # ลบแถวว่างทั้งหมด
    df = df.dropna(how='all')

    df_list.append(df)

# รวมทุกชีทเข้าด้วยกัน
patients = pd.concat(df_list, ignore_index=True)

# ลบข้อมูลซ้ำ
patients = patients.drop_duplicates()

# ตรวจสอบขนาดข้อมูลสุดท้าย
print(f"✅ รวมข้อมูลผู้ป่วยสำเร็จ: {patients.shape[0]} แถว, {patients.shape[1]} คอลัมน์")

# แสดงตัวอย่างข้อมูล

print(patients.head())


In [ ]:
weather = pd.read_excel("rain_all_years_sorted3.xlsx")
weather['วัน'] = pd.to_datetime(weather['วัน'])


In [ ]:
def get_rolling_avg(date, n_days=15):
    start_date = date - pd.Timedelta(days=n_days)
    mask = (weather['วัน'] >= start_date) & (weather['วัน'] < date)
    subset = weather.loc[mask]
    if subset.empty:
        return pd.Series([None, None, None])
    return pd.Series([
        subset['อุณหภูมิ'].mean(),
        subset['ฝน'].mean(),
        subset['ชื้น'].mean()
    ])

# เพิ่มคอลัมน์ค่าเฉลี่ยย้อนหลัง 15 วัน
patients[['temp_15d_avg', 'rain_15d_avg', 'humid_15d_avg']] = patients['วันเริ่มป่วย'].apply(get_rolling_avg)


In [ ]:
# สร้างฟีเจอร์วันที่เป็นรายวัน
patients['date'] = patients['วันเริ่มป่วย'].dt.date

# จำนวนผู้ป่วยต่อวัน
daily_counts = patients.groupby('date').size().reset_index(name='cases')

# ค่ากลางของตัวแปรรายวัน (รวม weather)
daily_weather = patients.groupby('date').agg({
    'temp_15d_avg': 'mean',
    'rain_15d_avg': 'mean',
    'humid_15d_avg': 'mean',
    'อายุ(ปี)': 'mean',
    'เพศ': lambda x: x.mode()[0] if not x.mode().empty else None,
    'อาชีพ': lambda x: x.mode()[0] if not x.mode().empty else None,
    'ตำบล': lambda x: x.mode()[0] if not x.mode().empty else None,
    'อำเภอ': lambda x: x.mode()[0] if not x.mode().empty else None,
    'เดือน': lambda x: x.mode()[0] if not x.mode().empty else None,
    'ปี': lambda x: x.mode()[0] if not x.mode().empty else None,
    'ไตรมาส': lambda x: x.mode()[0] if not x.mode().empty else None,
    'ฤดูกาล': lambda x: x.mode()[0] if not x.mode().empty else None
}).reset_index()

# รวมข้อมูลทั้งหมด
df_final = pd.merge(daily_counts, daily_weather, on='date', how='left')
print("✅ ได้ข้อมูลรวมพร้อมใช้งานสำหรับโมเดล:", df_final.shape)


In [ ]:
print("ข้อมูลสุดท้าย df_final shape:", df_final.shape)
print("ตัวอย่างข้อมูล:")
print(df_final.head())
print("ข้อมูล missing:")
print(df_final.isnull().sum())


In [ ]:
import pandas as pd

# โหลดข้อมูลสภาพอากาศจากทุกชีท
file_path = 'rain.xlsx'
xls = pd.ExcelFile(file_path)

df_list = []
for sheet in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet)
    df = df.rename(columns={'ปี': 'year', 'เดือน': 'month', 'วันที่': 'day'})

    df['year'] = pd.to_numeric(df['year'], errors='coerce')
    df['month'] = pd.to_numeric(df['month'], errors='coerce')
    df['day'] = pd.to_numeric(df['day'], errors='coerce')
    df = df.dropna(subset=['year', 'month', 'day'])

    df['วัน'] = pd.to_datetime(df[['year', 'month', 'day']], errors='coerce')
    df = df.dropna(subset=['วัน'])

    for col in ['อุณหภูมิ', 'ฝน', 'ชื้น']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    df = df[df[['อุณหภูมิ', 'ฝน', 'ชื้น']].notna().all(axis=1)]
    df_list.append(df)

weather = pd.concat(df_list, ignore_index=True)
weather = weather.drop_duplicates()
weather = weather.sort_values('วัน').reset_index(drop=True)

# บันทึกข้อมูลสภาพอากาศที่ clean แล้ว
weather.to_excel('rain_all_years_sorted3.xlsx', index=False)
print("✅ บันทึก rain_all_years_sorted3.xlsx เรียบร้อยแล้ว")

# โหลดข้อมูลผู้ป่วย
xls = pd.ExcelFile('DHFcc12.xlsx')
df_list = []

for sheet in xls.sheet_names:
    if sheet == "Table1":
        continue

    df = pd.read_excel(xls, sheet_name=sheet)
    df.columns = df.columns.str.strip()

    if 'วันเริ่มป่วย' in df.columns:
        df['วันเริ่มป่วย'] = pd.to_datetime(df['วันเริ่มป่วย'], errors='coerce')

    for col in ['อายุ(ปี)', 'อายุ(เดือน)']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    df = df.dropna(subset=['วันเริ่มป่วย'])
    df = df.dropna(how='all')

    df_list.append(df)

patients = pd.concat(df_list, ignore_index=True)
patients = patients.drop_duplicates()
print(f"✅ รวมข้อมูลผู้ป่วยสำเร็จ: {patients.shape[0]} แถว, {patients.shape[1]} คอลัมน์")

# โหลด weather อีกครั้งเพื่อความชัวร์
weather = pd.read_excel("rain_all_years_sorted3.xlsx")
weather['วัน'] = pd.to_datetime(weather['วัน'])

# ฟังก์ชัน rolling average
def get_rolling_avg(date, n_days=15):
    start_date = date - pd.Timedelta(days=n_days)
    mask = (weather['วัน'] >= start_date) & (weather['วัน'] < date)
    subset = weather.loc[mask]
    if subset.empty:
        return pd.Series([None, None, None])
    return pd.Series([
        subset['อุณหภูมิ'].mean(),
        subset['ฝน'].mean(),
        subset['ชื้น'].mean()
    ])

# คำนวณ rolling average ย้อนหลัง 15 วัน
patients[['temp_15d_avg', 'rain_15d_avg', 'humid_15d_avg']] = patients['วันเริ่มป่วย'].apply(get_rolling_avg)
print("✅ คำนวณ rolling average เสร็จ")

# สร้างฟีเจอร์วัน/เดือน/ไตรมาส/ฤดูกาล
patients['ปี'] = patients['วันเริ่มป่วย'].dt.year
patients['เดือน'] = patients['วันเริ่มป่วย'].dt.month
patients['ไตรมาส'] = patients['วันเริ่มป่วย'].dt.quarter

def assign_season(row):
    month = row['เดือน']
    day = row['วันเริ่มป่วย'].day
    if (month == 2 and day >= 15) or month in [3, 4] or (month == 5 and day < 15):
        return 'Summer'
    elif (month == 5 and day >= 15) or month in [6, 7, 8, 9] or (month == 10 and day < 15):
        return 'Rainy'
    else:
        return 'Winter'

patients['ฤดูกาล'] = patients.apply(assign_season, axis=1)

# รวมข้อมูลรายวัน
patients['date'] = patients['วันเริ่มป่วย'].dt.date
daily_counts = patients.groupby('date').size().reset_index(name='cases')

daily_weather = patients.groupby('date').agg({
    'temp_15d_avg': 'mean',
    'rain_15d_avg': 'mean',
    'humid_15d_avg': 'mean',
    'อายุ(ปี)': 'mean',
    'เพศ': lambda x: x.mode()[0] if not x.mode().empty else None,
    'อาชีพ': lambda x: x.mode()[0] if not x.mode().empty else None,
    'ตำบล': lambda x: x.mode()[0] if not x.mode().empty else None,
    'อำเภอ': lambda x: x.mode()[0] if not x.mode().empty else None,
    'เดือน': lambda x: x.mode()[0] if not x.mode().empty else None,
    'ปี': lambda x: x.mode()[0] if not x.mode().empty else None,
    'ไตรมาส': lambda x: x.mode()[0] if not x.mode().empty else None,
    'ฤดูกาล': lambda x: x.mode()[0] if not x.mode().empty else None
}).reset_index()

# รวมข้อมูลทั้งหมด
df_final = pd.merge(daily_counts, daily_weather, on='date', how='left')
print("✅ รวมข้อมูล df_final สำเร็จ:", df_final.shape)

# จัดการ missing
missing_before = df_final.isnull().sum().sum()
print(f"⚠️ พบ missing values: {missing_before} ช่อง")

df_final_clean = df_final.dropna()
print("✅ ลบ missing rows แล้ว → ขนาดข้อมูล:", df_final_clean.shape)

# บันทึกข้อมูลที่สะอาดแล้ว
df_final_clean.to_csv("df_final_clean.csv", index=False)
df_final_clean.to_excel("df_final_clean.xlsx", index=False)
print("📁 บันทึกไฟล์ df_final_clean.csv และ .xlsx เรียบร้อย")


In [ ]:
# บันทึกข้อมูลที่สะอาดแล้ว
df_final_clean.to_csv("df_final_clean.csv", index=False)
df_final_clean.to_excel("df_final_clean.xlsx", index=False)
print("📁 บันทึกไฟล์ df_final_clean.csv และ .xlsx เรียบร้อย")

In [ ]:
print(df.columns)


In [ ]:
# แก้ชื่อคอลัมน์ให้เป็น ASCII-friendly (หรือใช้ชื่ออังกฤษทั้งหมด)
df_clean = df_clean.rename(columns={
    'เพศ': 'gender',
    'อาชีพ': 'occupation',
    'ตำบล': 'subdistrict',
    'อำเภอ': 'district',
    'เดือน': 'month',
    'ไตรมาส': 'quarter'
})


In [ ]:
import pandas as pd
import scipy.stats as stats
from scipy.stats import spearmanr, pointbiserialr, kruskal
import statsmodels.api as sm
import statsmodels.formula.api as smf

# -----------------------------
# ✅ 0. Load and Rename Columns
# -----------------------------
df = pd.read_csv("df_final_clean.csv")  # ใส่ path ตามจริง
df_clean = df.copy()

# เปลี่ยนชื่อคอลัมน์ให้เป็นภาษาอังกฤษเพื่อใช้ในสูตร model
df_clean = df_clean.rename(columns={
    'เพศ': 'gender',
    'อาชีพ': 'occupation',
    'ตำบล': 'subdistrict',
    'อำเภอ': 'district',
    'เดือน': 'month',
    'ไตรมาส': 'quarter',
    'อายุ(ปี)': 'age',
    'ฤดูกาล': 'season',

})

# -----------------------------
# ✅ 1. วิเคราะห์เชิงเดี่ยว
# -----------------------------
target = 'cases'
numeric_vars = ['temp_15d_avg', 'rain_15d_avg', 'humid_15d_avg']  # ตัด month_name ออก เพราะไม่มีคอลัมน์นี้
binary_cats = ['gender']
multi_cats = ['occupation', 'subdistrict', 'district', 'month', 'season']
  # ใช้ month แทน month_name

# แปลง gender เป็นตัวเลข (0/1)
if 'gender' in df_clean.columns:
    df_clean['gender_code'] = df_clean['gender'].map({'ชาย': 0, 'หญิง': 1})

results = []

# 1.1 Spearman correlation (numeric)
for var in numeric_vars:
    corr, p = spearmanr(df_clean[var], df_clean[target])
    results.append({
        'Variable': var,
        'Test': 'Spearman Correlation',
        'Stat': round(corr, 3),
        'P-Value': round(p, 4)
    })

# 1.2 Point-Biserial (binary)
for var in binary_cats:
    encoded_var = var + '_code'
    if encoded_var in df_clean.columns and df_clean[encoded_var].nunique() == 2:
        corr, p = pointbiserialr(df_clean[encoded_var], df_clean[target])
        results.append({
            'Variable': var,
            'Test': 'Point-Biserial',
            'Stat': round(corr, 3),
            'P-Value': round(p, 4)
        })

# 1.3 Kruskal-Wallis (multi-category)
for var in multi_cats:
    df_non_null = df_clean.dropna(subset=[var])
    groups = [group[target].dropna() for name, group in df_non_null.groupby(var)]
    if len(groups) > 1:
        h_stat, p = kruskal(*groups)
        results.append({
            'Variable': var,
            'Test': 'Kruskal-Wallis',
            'Stat': round(h_stat, 3),
            'P-Value': round(p, 4)
        })

# แสดงผล
results_df = pd.DataFrame(results).sort_values('P-Value')
print("\n📊 ผลวิเคราะห์เชิงเดี่ยว (เรียงตาม p-value):")
print(results_df)

# -----------------------------
# ✅ 2. Negative Binomial Regression
# -----------------------------
print("\n📈 ผลการวิเคราะห์ Negative Binomial Regression:")

formula = """
cases ~ temp_15d_avg + rain_15d_avg + humid_15d_avg
+ C(gender) + C(occupation) + C(subdistrict) + C(month) + C(season)
"""


nb_model = smf.glm(
    formula=formula,
    data=df_clean,
    family=sm.families.NegativeBinomial()
).fit()

print(nb_model.summary())


In [ ]:
# รันโมเดล Negative Binomial และเก็บผลลัพธ์
nb_model = smf.glm(
    formula=formula,
    data=df_clean,
    family=sm.families.NegativeBinomial()
).fit()

# แสดงผลสรุปโมเดล
print(nb_model.summary())

# ดึงตารางผลลัพธ์ coefficients
summary_df = nb_model.summary2().tables[1]

# กรองเฉพาะตัวแปรที่มีนัยสำคัญ p-value < 0.05
significant_vars = summary_df[summary_df['P>|z|'] < 0.05]

print("\n📌 ตัวแปรที่มีนัยสำคัญ (p-value < 0.05):")
print(significant_vars[['Coef.', 'Std.Err.', 'z', 'P>|z|']])

# ตรวจสอบ overdispersion ของข้อมูล
mean_cases = df_clean['cases'].mean()
var_cases = df_clean['cases'].var()

print(f"\nMean of cases: {mean_cases:.4f}")
print(f"Variance of cases: {var_cases:.4f}")

if var_cases > mean_cases:
    print("📈 มี overdispersion → เหมาะกับ Negative Binomial")
else:
    print("📉 ไม่มี overdispersion → อาจไม่เหมาะกับ Negative Binomial")


In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

# โหลดข้อมูล
df = pd.read_csv("df_final_clean.csv")
df_clean = df.copy()

# เปลี่ยนชื่อคอลัมน์
df_clean = df_clean.rename(columns={
    'เพศ': 'gender',
    'อาชีพ': 'occupation',
    'ตำบล': 'subdistrict',
    'อำเภอ': 'district',
    'เดือน': 'month',
    'ไตรมาส': 'quarter',
    'อายุ(ปี)': 'age',
    'ฤดูกาล': 'season',
})

# กำหนดสูตรโมเดล Negative Binomial
formula = """
cases ~ temp_15d_avg + rain_15d_avg + humid_15d_avg
+ C(gender) + C(occupation) + C(subdistrict) + C(month) + C(season)
"""

# รันโมเดล Negative Binomial
nb_model = smf.glm(
    formula=formula,
    data=df_clean,
    family=sm.families.NegativeBinomial()
).fit()

# แสดงผล summary
print(nb_model.summary())

# ดึงตาราง coefficients
summary_df = nb_model.summary2().tables[1]

# กรองตัวแปรที่ p-value < 0.05
significant_vars = summary_df[summary_df['P>|z|'] < 0.05]

print("\n📌 ตัวแปรที่มีนัยสำคัญ (p-value < 0.05):")
print(significant_vars[['Coef.', 'Std.Err.', 'z', 'P>|z|']])

# ตรวจสอบ overdispersion
mean_cases = df_clean['cases'].mean()
var_cases = df_clean['cases'].var()
print(f"\nMean of cases: {mean_cases}")
print(f"Variance of cases: {var_cases}")

if var_cases > mean_cases:
    print("📈 มี overdispersion → เหมาะกับ Negative Binomial")
else:
    print("📉 ไม่มี overdispersion → อาจไม่เหมาะกับ Negative Binomial")

# เตรียมข้อมูลสำหรับคำนวณ VIF (แปลง categorical เป็น dummy)
X = pd.get_dummies(df_clean[['temp_15d_avg', 'rain_15d_avg', 'humid_15d_avg',
                             'gender', 'occupation', 'subdistrict', 'month', 'season']], drop_first=True)
# เติมค่า missing ถ้ามี (โดยใช้ mean)
X = X.fillna(X.mean())

# เพิ่ม constant term
X = sm.add_constant(X)

# แปลงข้อมูลเป็น float ทั้งหมดก่อนคำนวณ VIF
X = X.astype(float)

# คำนวณ VIF
vif_data = pd.DataFrame()
vif_data['feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print("\n📊 Variance Inflation Factor (VIF):")
print(vif_data.sort_values(by='VIF', ascending=False))



In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

# 1. โหลดข้อมูลและเปลี่ยนชื่อคอลัมน์
df = pd.read_csv("df_final_clean.csv")
df_clean = df.copy()
df_clean = df_clean.rename(columns={
    'เพศ': 'gender',
    'อาชีพ': 'occupation',
    'ตำบล': 'subdistrict',
    'อำเภอ': 'district',
    'เดือน': 'month',
    'ไตรมาส': 'quarter',
    'อายุ(ปี)': 'age',
    'ฤดูกาล': 'season',
})

# 2. แปลง categorical เป็น category dtype
for col in ['gender', 'occupation', 'subdistrict', 'district', 'month', 'season']:
    df_clean[col] = df_clean[col].astype('category')

# 3. สร้างสูตรโมเดลใหม่ ลด occupation ออกเพื่อลด multicollinearity
formula_reduced = """
cases ~ rain_15d_avg + humid_15d_avg
+ C(subdistrict) + C(season)
"""

# 4. Fit Negative Binomial model ด้วยสูตรใหม่
nb_model_reduced = smf.glm(
    formula=formula_reduced,
    data=df_clean,
    family=sm.families.NegativeBinomial()
).fit()

# 5. แสดงผลสรุปโมเดล
print(nb_model_reduced.summary())

# 6. ดึงตารางผลลัพธ์ coefficients
summary_df = nb_model_reduced.summary2().tables[1]

# 7. กรองเฉพาะตัวแปรที่ p-value < 0.05
significant_vars = summary_df[summary_df['P>|z|'] < 0.05]

print("\n📌 ตัวแปรที่มีนัยสำคัญ (p-value < 0.05):")
print(significant_vars[['Coef.', 'Std.Err.', 'z', 'P>|z|']])

# 8. ตรวจสอบ overdispersion
mean_cases = df_clean['cases'].mean()
var_cases = df_clean['cases'].var()
print(f"\nMean of cases: {mean_cases}")
print(f"Variance of cases: {var_cases}")

if var_cases > mean_cases:
    print("📈 มี overdispersion → เหมาะกับ Negative Binomial")
else:
    print("📉 ไม่มี overdispersion → อาจไม่เหมาะกับ Negative Binomial")


In [ ]:
# ดึงชื่อของตัวแปรที่มีนัยสำคัญ ยกเว้น Intercept
selected_features = [var for var in significant_vars.index if var != 'Intercept']

# สร้างชุดข้อมูลใหม่เฉพาะตัวแปรที่สำคัญ
# ต้องแปลง C(...) เป็นชื่อคอลัมน์จริง
X = pd.get_dummies(df_clean, drop_first=True)

# กรองเฉพาะคอลัมน์ที่ตรงกับ selected_features
X_selected = X[[col for col in X.columns if any(f in col for f in selected_features)]]

# เป้าหมาย (Target)
y = df_clean['cases']


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# เลือกเฉพาะ features ที่เกี่ยวข้อง
features = ['rain_15d_avg', 'humid_15d_avg', 'subdistrict', 'season']
df_ml = df_clean[features + ['cases']].dropna()

# One-hot encode
df_ml = pd.get_dummies(df_ml, drop_first=True)

X = df_ml.drop('cases', axis=1)
y = df_ml['cases']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.4f}")
print(f"R² Score: {r2:.4f}")


In [ ]:
pip install xgboost scikit-learn


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder

# 1. โหลดและเตรียมข้อมูล
df = pd.read_csv("df_final_clean.csv")

# เปลี่ยนชื่อคอลัมน์
df = df.rename(columns={
    'เพศ': 'gender',
    'อาชีพ': 'occupation',
    'ตำบล': 'subdistrict',
    'อำเภอ': 'district',
    'เดือน': 'month',
    'ไตรมาส': 'quarter',
    'อายุ(ปี)': 'age',
    'ฤดูกาล': 'season',
})

# เลือกเฉพาะตัวแปรที่มีนัยสำคัญ
features = ['subdistrict', 'season', 'rain_15d_avg', 'humid_15d_avg']
target = 'cases'

df_model = df[features + [target]].copy()

# One-hot encode ตัวแปร category
cat_cols = ['subdistrict', 'season']
df_model = pd.get_dummies(df_model, columns=cat_cols, drop_first=True)

# 2. แบ่งข้อมูล
X = df_model.drop(columns=[target])
y = df_model[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. สร้างโมเดล
models = {
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42),
    "Linear Regression": LinearRegression()
}

# 4. เทรนและประเมินผล
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    from sklearn.metrics import mean_squared_error

    mse = mean_squared_error(y_test, y_pred)
    rmse = mse ** 0.5

    print(f"\n📊 ผลการประเมินโมเดล {name}:")
    print(f"RMSE: {rmse:.4f}")
    print(f"R² Score: {r2:.4f}")


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb

# เตรียมข้อมูล
df_xgb = df_clean.copy()

features = ['subdistrict', 'month', 'rain_15d_avg', 'humid_15d_avg']
target = 'cases'

X = df_xgb[features]
y = df_xgb[target]

# แปลง categorical เป็น dummy variables
X_encoded = pd.get_dummies(X, columns=['subdistrict', 'month'], drop_first=True)

# แบ่งข้อมูล train/test
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.3, random_state=42
)

# สร้าง DMatrix สำหรับ XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# กำหนดพารามิเตอร์ XGBoost
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'seed': 42
}

# Train โมเดล
num_rounds = 100
xgb_model = xgb.train(params, dtrain, num_rounds)

# ทำนาย
y_pred = xgb_model.predict(dtest)

# ประเมินผล
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse:.3f}")
print(f"R-squared (R2): {r2:.3f}")

# ดูความสำคัญของตัวแปร
import matplotlib.pyplot as plt

xgb.plot_importance(xgb_model, max_num_features=10)
plt.show()


In [ ]:
อีกอัน


In [ ]:
import pandas as pd
import scipy.stats as stats
from scipy.stats import pointbiserialr

# ลิสต์ตัวแปร
numeric_vars = ['temp_15d_avg', 'rain_15d_avg', 'humid_15d_avg', 'อายุ(ปี)']
binary_cats = ['เพศ']
multi_cats = ['อาชีพ', 'ตำบล', 'อำเภอ', 'เดือน', 'ไตรมาส']

# ตัวแปรเป้าหมาย
target = 'cases'

# สำเนา df เพื่อตัด NaN
df_clean = df_final.copy()

# แปลง 'เพศ' เป็นตัวเลข (เช่น 'ชาย' = 0, 'หญิง' = 1)
if 'เพศ' in df_clean.columns:
    df_clean['เพศ_รหัส'] = df_clean['เพศ'].map({'ชาย': 0, 'หญิง': 1})

# เก็บผลลัพธ์
results = []

# 1. Pearson correlation สำหรับ numeric
for var in numeric_vars:
    corr, p = stats.pearsonr(df_clean[var], df_clean[target])
    results.append({
        'Variable': var,
        'Test': 'Pearson',
        'Stat': round(corr, 3),
        'P-Value': round(p, 4)
    })

# 2. Point-Biserial สำหรับ binary (หลัง encode แล้ว)
for var in binary_cats:
    encoded_var = var + '_รหัส'
    if encoded_var in df_clean.columns and df_clean[encoded_var].nunique() == 2:
        corr, p = pointbiserialr(df_clean[encoded_var], df_clean[target])
        results.append({
            'Variable': var,
            'Test': 'Point-Biserial',
            'Stat': round(corr, 3),
            'P-Value': round(p, 4)
        })

# 3. ANOVA สำหรับ multi-category
for var in multi_cats:
    df_non_null = df_clean.dropna(subset=[var])
    groups = [group[target].dropna() for name, group in df_non_null.groupby(var)]
    if len(groups) > 1:
        f_stat, p = stats.f_oneway(*groups)
        results.append({
            'Variable': var,
            'Test': 'ANOVA',
            'Stat': round(f_stat, 3),
            'P-Value': round(p, 4)
        })

# แสดงผลลัพธ์
results_df = pd.DataFrame(results).sort_values('P-Value')
print("\n📊 สรุปผลการทดสอบความสัมพันธ์ (เรียงตาม p-value):")
print(results_df)


In [ ]:
# กรองเฉพาะตัวแปรที่มีนัยสำคัญ
significant_vars = results_df[results_df['P-Value'] < 0.05]['Variable'].tolist()

# แสดงผล
print("✅ ตัวแปรที่ผ่านการคัดเลือก:", significant_vars)


In [ ]:
print(df_selected.head())


In [ ]:
X = df[significant_vars]
y = df['cases']


In [ ]:
import pandas as pd

# สมมติ df_selected คือ dataframe ที่ได้จากการเลือกตัวแปรแล้ว
# แยก features และ target
X = df_selected.drop(columns=['cases'])
y = df_selected['cases']

# หาคอลัมน์ที่เป็น categorical
cat_cols = X.select_dtypes(include=['object', 'category']).columns

# ทำ One-Hot Encoding
X_encoded = pd.get_dummies(X, columns=cat_cols, drop_first=True)

print(X_encoded.head())


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

# สร้างและฝึกโมเดล
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

# ทำนายข้อมูลทดสอบ
y_pred_rf = rf.predict(X_test)

# คำนวณ metric ต่าง ๆ
rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2 = r2_score(y_test, y_pred_rf)
mae = mean_absolute_error(y_test, y_pred_rf)

print(f"RandomForest R2: {r2:.4f}")
print(f"RandomForest RMSE: {rmse:.4f}")
print(f"RandomForest MAE: {mae:.4f}")


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# กำหนดพารามิเตอร์ที่ต้องการลอง
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

rf = RandomForestRegressor(random_state=42)

# ใช้ RandomizedSearchCV เพื่อหาพารามิเตอร์ที่ดีที่สุด
search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=20,  # จำนวนรอบสุ่มลองพารามิเตอร์
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

search.fit(X_train, y_train)

# โมเดลที่ดีที่สุด
best_rf = search.best_estimator_

# ทำนายข้อมูลทดสอบ
y_pred_rf = best_rf.predict(X_test)

# คำนวณ metric
rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2 = r2_score(y_test, y_pred_rf)
mae = mean_absolute_error(y_test, y_pred_rf)

print(f"Best Parameters: {search.best_params_}")
print(f"RandomForest R2: {r2:.4f}")
print(f"RandomForest RMSE: {rmse:.4f}")
print(f"RandomForest MAE: {mae:.4f}")


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# กำหนดพารามิเตอร์ที่ต้องการลอง
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

rf = RandomForestRegressor(random_state=42)

# ใช้ RandomizedSearchCV เพื่อหาพารามิเตอร์ที่ดีที่สุด
search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=20,  # จำนวนรอบสุ่มลองพารามิเตอร์
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

search.fit(X_train, y_train)

# โมเดลที่ดีที่สุด
best_rf = search.best_estimator_

# ทำนายข้อมูลทดสอบ
y_pred_rf = best_rf.predict(X_test)

# คำนวณ metric
rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2 = r2_score(y_test, y_pred_rf)
mae = mean_absolute_error(y_test, y_pred_rf)

print(f"Best Parameters: {search.best_params_}")
print(f"RandomForest R2: {r2:.4f}")
print(f"RandomForest RMSE: {rmse:.4f}")
print(f"RandomForest MAE: {mae:.4f}")


# XGBOOST

In [ ]:
pip install xgboost


In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# ✅ ตัวแปรที่เลือกไว้
selected_features = ['ตำบล', 'อำเภอ', 'ไตรมาส', 'เดือน', 'อาชีพ', 'temp_15d_avg', 'rain_15d_avg']

# ✅ เตรียม X, y
X = df[selected_features].copy()
y = df['cases']

# ✅ interaction term
X['ฝน_อุณหภูมิ'] = X['temp_15d_avg'] * X['rain_15d_avg']

# ✅ แบ่ง train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Categorical & numeric columns
categorical_cols = ['ตำบล', 'อำเภอ', 'ไตรมาส', 'เดือน', 'อาชีพ']
numeric_cols = ['temp_15d_avg', 'rain_15d_avg']

# ✅ Preprocessing
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
], remainder='passthrough')  # numeric ผ่านตรง

# ✅ สร้าง Pipeline
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', xgb_model)
])

# ✅ Hyperparameter tuning
param_dist = {
    'regressor__n_estimators': [100, 200, 300, 500],
    'regressor__max_depth': [4, 6, 8, 10],
    'regressor__learning_rate': [0.03, 0.05, 0.07],
    'regressor__subsample': [0.6, 0.7, 0.8],
    'regressor__colsample_bytree': [0.6, 0.8, 1.0],
    'regressor__gamma': [0, 0.1, 0.3],
    'regressor__reg_alpha': [0, 0.01, 0.1],
    'regressor__reg_lambda': [1, 1.5, 2]
}

search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter=150,
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

# ✅ เทรน
search.fit(X_train, y_train)

# ✅ ทำนายและประเมินผล
best_model = search.best_estimator_
y_pred = best_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"✅ Best Parameters: {search.best_params_}")
print(f"✅ XGBoost R2: {r2:.4f}")
print(f"✅ XGBoost RMSE: {rmse:.4f}")
print(f"✅ XGBoost MAE: {mae:.4f}")


In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# ✅ ตัวแปรที่เลือกไว้
selected_features = ['ตำบล', 'อำเภอ', 'ไตรมาส', 'เดือน', 'อาชีพ', 'temp_15d_avg', 'rain_15d_avg']

# ✅ เตรียม X, y
X = df[selected_features].copy()
y = df['cases']

# ✅ interaction term
X['ฝน_อุณหภูมิ'] = X['temp_15d_avg'] * X['rain_15d_avg']

# ✅ แบ่ง train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Categorical & numeric columns
categorical_cols = ['ตำบล', 'อำเภอ', 'ไตรมาส', 'เดือน', 'อาชีพ']
numeric_cols = ['temp_15d_avg', 'rain_15d_avg', 'ฝน_อุณหภูมิ']

# ✅ Preprocessing
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
], remainder='passthrough')  # numeric ผ่านตรง

# ✅ สร้าง Pipeline
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', xgb_model)
])

# ✅ Hyperparameter tuning
param_dist = {
    'regressor__n_estimators': [100, 200, 300, 500],
    'regressor__max_depth': [4, 6, 8, 10],
    'regressor__learning_rate': [0.03, 0.05, 0.07],
    'regressor__subsample': [0.6, 0.7, 0.8],
    'regressor__colsample_bytree': [0.6, 0.8, 1.0],
    'regressor__gamma': [0, 0.1, 0.3],
    'regressor__reg_alpha': [0, 0.01, 0.1],
    'regressor__reg_lambda': [1, 1.5, 2]
}

search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter=50,
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

# ✅ เทรน
search.fit(X_train, y_train)

# ✅ ทำนายและประเมินผล
best_model = search.best_estimator_
y_pred = best_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"✅ Best Parameters: {search.best_params_}")
print(f"✅ XGBoost R2: {r2:.4f}")
print(f"✅ XGBoost RMSE: {rmse:.4f}")
print(f"✅ XGBoost MAE: {mae:.4f}")


## deep learning

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

# สร้างโมเดลง่าย ๆ
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# ฝึกโมเดล
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# ทำนาย
y_pred_dl = model.predict(X_test).flatten()

# คำนวณ metric
rmse_dl = np.sqrt(mean_squared_error(y_test, y_pred_dl))
r2_dl = r2_score(y_test, y_pred_dl)
mae_dl = mean_absolute_error(y_test, y_pred_dl)

print(f"Deep Learning R2: {r2_dl:.4f}")
print(f"Deep Learning RMSE: {rmse_dl:.4f}")
print(f"Deep Learning MAE: {mae_dl:.4f}")


In [ ]:
import pandas as pd

# โหลดข้อมูล CSV (เปลี่ยนชื่อไฟล์ตามของคุณ)
df = pd.read_csv('df_final_clean.csv')

# แปลงคอลัมน์ date เป็น datetime
df['date'] = pd.to_datetime(df['date'])

# ตั้ง date เป็น index ของ dataframe
df = df.set_index('date')

print(df.head())


In [ ]:
import matplotlib.pyplot as plt

df['cases'].plot(figsize=(12,6), title='จำนวนเคสตามเวลา')
plt.show()


In [ ]:
train_size = int(len(df) * 0.8)
train = df.iloc[:train_size]
test = df.iloc[train_size:]


In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

model = SARIMAX(train['cases'], order=(1,1,1), seasonal_order=(1,1,1,12))
result = model.fit()

print(result.summary())


In [ ]:
pred = result.predict(start=len(train), end=len(train) + len(test) - 1)


In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

rmse = np.sqrt(mean_squared_error(test['cases'], pred))
print(f'RMSE: {rmse:.4f}')

plt.figure(figsize=(12,6))
plt.plot(train.index, train['cases'], label='Train')
plt.plot(test.index, test['cases'], label='Test')
plt.plot(pred.index, pred, label='Predicted')
plt.legend()
plt.title('ผลการทำนายจำนวนเคส')
plt.show()


# ขั้นตอน 1: โหลดและเตรียมข้อมูล

In [ ]:
import pandas as pd

# โหลดข้อมูลสภาพอากาศจากหลายชีทในไฟล์ rain.xlsx
file_path = 'rain.xlsx'
xls = pd.ExcelFile(file_path)

df_list = []
for sheet in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet)
    df = df.rename(columns={'ปี': 'year', 'เดือน': 'month', 'วันที่': 'day'})

    # แปลงเป็นตัวเลข
    df['year'] = pd.to_numeric(df['year'], errors='coerce')
    df['month'] = pd.to_numeric(df['month'], errors='coerce')
    df['day'] = pd.to_numeric(df['day'], errors='coerce')
    df = df.dropna(subset=['year', 'month', 'day'])

    # สร้าง datetime column
    df['วัน'] = pd.to_datetime(df[['year', 'month', 'day']], errors='coerce')
    df = df.dropna(subset=['วัน'])

    # แปลงข้อมูลสภาพอากาศให้เป็นตัวเลข
    for col in ['อุณหภูมิ', 'ฝน', 'ชื้น']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # กรองเอาเฉพาะแถวที่มีข้อมูลครบ
    df = df[df[['อุณหภูมิ', 'ฝน', 'ชื้น']].notna().all(axis=1)]
    df_list.append(df)

# รวมข้อมูลสภาพอากาศทั้งหมด
weather = pd.concat(df_list, ignore_index=True).drop_duplicates().sort_values('วัน').reset_index(drop=True)

# บันทึกข้อมูลสภาพอากาศที่ clean แล้ว
weather.to_excel('rain_all_years_sorted3.xlsx', index=False)
print("✅ บันทึก rain_all_years_sorted3.xlsx เรียบร้อยแล้ว")


# ขั้นตอน 2: โหลดและเตรียมข้อมูลผู้ป่วย

In [ ]:
# โหลดข้อมูลผู้ป่วยจากหลายชีทในไฟล์ DHFcc12.xlsx
xls = pd.ExcelFile('DHFcc12.xlsx')
df_list = []

for sheet in xls.sheet_names:
    if sheet == "Table1":  # ข้ามชีท Table1
        continue

    df = pd.read_excel(xls, sheet_name=sheet)
    df.columns = df.columns.str.strip()

    if 'วันเริ่มป่วย' in df.columns:
        df['วันเริ่มป่วย'] = pd.to_datetime(df['วันเริ่มป่วย'], errors='coerce')

    for col in ['อายุ(ปี)', 'อายุ(เดือน)']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    df = df.dropna(subset=['วันเริ่มป่วย'])
    df = df.dropna(how='all')

    df_list.append(df)

patients = pd.concat(df_list, ignore_index=True).drop_duplicates()
print(f"✅ รวมข้อมูลผู้ป่วยสำเร็จ: {patients.shape[0]} แถว, {patients.shape[1]} คอลัมน์")


# ขั้นตอน 3: คำนวณ rolling average ของสภาพอากาศย้อนหลัง 15 วัน สำหรับแต่ละวันเริ่มป่วย

In [ ]:
# โหลดข้อมูลสภาพอากาศที่จัดการแล้ว (เพื่อความแน่นอน)
weather = pd.read_excel("rain_all_years_sorted3.xlsx")
weather['วัน'] = pd.to_datetime(weather['วัน'])

# ฟังก์ชันคำนวณ rolling average ย้อนหลัง 15 วัน
def get_rolling_avg(date, n_days=15):
    start_date = date - pd.Timedelta(days=n_days)
    mask = (weather['วัน'] >= start_date) & (weather['วัน'] < date)
    subset = weather.loc[mask]
    if subset.empty:
        return pd.Series([None, None, None])
    return pd.Series([
        subset['อุณหภูมิ'].mean(),
        subset['ฝน'].mean(),
        subset['ชื้น'].mean()
    ])

# คำนวณ rolling average สำหรับผู้ป่วยแต่ละราย
patients[['temp_15d_avg', 'rain_15d_avg', 'humid_15d_avg']] = patients['วันเริ่มป่วย'].apply(get_rolling_avg)
print("✅ คำนวณ rolling average เสร็จ")


# ขั้นตอน 4: สร้างฟีเจอร์วันที่ เช่น ปี, เดือน, ไตรมาส, ฤดูกาล

In [ ]:
patients['ปี'] = patients['วันเริ่มป่วย'].dt.year
patients['เดือน'] = patients['วันเริ่มป่วย'].dt.month
patients['ไตรมาส'] = patients['วันเริ่มป่วย'].dt.quarter

def assign_season(row):
    month = row['เดือน']
    day = row['วันเริ่มป่วย'].day
    if (month == 2 and day >= 15) or month in [3,4] or (month == 5 and day < 15):
        return 'Summer'
    elif (month == 5 and day >= 15) or month in [6,7,8,9] or (month == 10 and day < 15):
        return 'Rainy'
    else:
        return 'Winter'

patients['ฤดูกาล'] = patients.apply(assign_season, axis=1)


# ขั้นตอน 5: รวมข้อมูลรายวันและคำนวณจำนวนผู้ป่วยรายวัน

In [ ]:
# สร้าง column วันแบบ date (ไม่มีเวลา)
patients['date'] = patients['วันเริ่มป่วย'].dt.date

# นับจำนวนผู้ป่วยแต่ละวัน (cases)
daily_counts = patients.groupby('date').size().reset_index(name='cases')

# คำนวณค่าเฉลี่ยและโหมดของตัวแปรอื่น ๆ ต่อวัน
daily_weather = patients.groupby('date').agg({
    'temp_15d_avg': 'mean',
    'rain_15d_avg': 'mean',
    'humid_15d_avg': 'mean',
    'อายุ(ปี)': 'mean',
    'เพศ': lambda x: x.mode()[0] if not x.mode().empty else None,
    'อาชีพ': lambda x: x.mode()[0] if not x.mode().empty else None,
    'ตำบล': lambda x: x.mode()[0] if not x.mode().empty else None,
    'อำเภอ': lambda x: x.mode()[0] if not x.mode().empty else None,
    'เดือน': lambda x: x.mode()[0] if not x.mode().empty else None,
    'ปี': lambda x: x.mode()[0] if not x.mode().empty else None,
    'ไตรมาส': lambda x: x.mode()[0] if not x.mode().empty else None,
    'ฤดูกาล': lambda x: x.mode()[0] if not x.mode().empty else None
}).reset_index()

# รวมข้อมูลผู้ป่วยรายวันกับข้อมูลสภาพอากาศ
df_final = pd.merge(daily_counts, daily_weather, on='date', how='left')
print("✅ รวมข้อมูล df_final สำเร็จ:", df_final.shape)


# ขั้นตอน 6: จัดการ missing values และบันทึกข้อมูล

In [ ]:
missing_before = df_final.isnull().sum().sum()
print(f"⚠️ พบ missing values: {missing_before} ช่อง")

df_final_clean = df_final.dropna()
print("✅ ลบ missing rows แล้ว → ขนาดข้อมูล:", df_final_clean.shape)

# บันทึกไฟล์ clean data
df_final_clean.to_csv("df_final_clean.csv", index=False)
df_final_clean.to_excel("df_final_clean.xlsx", index=False)
print("📁 บันทึกไฟล์ df_final_clean.csv และ df_final_clean.xlsx เรียบร้อย")


# ขั้นตอน 7: เปลี่ยนชื่อคอลัมน์เป็นภาษาอังกฤษ เพื่อใช้งานในโมเดล

In [ ]:
df_final_clean = pd.read_excel("/content/df_final_clean.xlsx")

In [ ]:
print(df_final_clean.isnull().sum())


In [ ]:
df_final_clean = df_final_clean.drop_duplicates() #ตรวจสอบข้อมูลซ้ำ (Duplicate Records)

In [ ]:
print(df_final_clean.dtypes) #ตรวจสอบประเภทข้อมูล (Data Types)


In [ ]:
df_clean = df_final_clean.rename(columns={
    'เพศ': 'gender',
    'อาชีพ': 'occupation',
    'ตำบล': 'subdistrict',
    'อำเภอ': 'district',
    'เดือน': 'month',
    'ไตรมาส': 'quarter',
    'อายุ(ปี)': 'age',
    'ฤดูกาล': 'season',
})


In [ ]:
print(df_clean.head())

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# โหลดข้อมูลจากไฟล์
df_final_clean = pd.read_excel("/content/df_final_clean.xlsx")

# ตัวอย่างการสร้างกราฟแท่ง (Bar Chart) แสดงการกระจายของ 'cases' ตาม 'season'
plt.figure(figsize=(10, 6))
df_final_clean.groupby('ฤดูกาล')['cases'].sum().plot(kind='bar', color='skyblue')
plt.title('Total Cases by Season')
plt.xlabel('Season')
plt.ylabel('Total Cases')
plt.xticks(rotation=45)
plt.show()

# ตัวอย่างการสร้างกราฟกระจาย (Scatter Plot) แสดงความสัมพันธ์ระหว่าง 'age' และ 'cases'
plt.figure(figsize=(10, 6))
plt.scatter(df_final_clean['อายุ(ปี)'], df_final_clean['cases'], color='orange')
plt.title('Scatter Plot of Age vs Cases')
plt.xlabel('Age')
plt.ylabel('Cases')
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# แปลงคอลัมน์ 'date' เป็น datetime
df_clean['date'] = pd.to_datetime(df_clean['date'], errors='coerce')

# กรองข้อมูลในช่วงปี 2017-2024
df_clean = df_clean[df_clean['date'].dt.year.between(2017, 2024)]

# สร้างคอลัมน์ 'month' และ 'year' เพื่อให้แยกกราฟตามเดือน
df_clean['month'] = df_clean['date'].dt.month
df_clean['year'] = df_clean['date'].dt.year

# คำนวณค่าเฉลี่ยตามปีและเดือน
avg_data = df_clean.groupby(['year', 'month']).agg({
    'temp_15d_avg': 'mean',
    'rain_15d_avg': 'mean',
    'humid_15d_avg': 'mean'
}).reset_index()

# ตั้งค่าการแสดงผล 3 กราฟ (temp_15d_avg, rain_15d_avg, humid_15d_avg) แยกตามเดือน
fig, axs = plt.subplots(3, 1, figsize=(12, 18), sharex=True)

# แสดงกราฟแค่เดือนที่จำเป็น (ทุกๆ 6 เดือน)
months_to_display = [1, 6, 12]
avg_data_filtered = avg_data[avg_data['month'].isin(months_to_display)]

# กราฟแสดงอุณหภูมิ (Temperature)
axs[0].plot(avg_data_filtered['year'].astype(str) + '-' + avg_data_filtered['month'].astype(str), avg_data_filtered['temp_15d_avg'], color='red', label='Temperature')
axs[0].set_title('Average Temperature (°C) from 2017 to 2024')
axs[0].set_ylabel('Temperature (°C)')
axs[0].legend()

# กราฟแสดงฝน (Rainfall)
axs[1].plot(avg_data_filtered['year'].astype(str) + '-' + avg_data_filtered['month'].astype(str), avg_data_filtered['rain_15d_avg'], color='green', label='Rainfall')
axs[1].set_title('Average Rainfall (mm) from 2017 to 2024')
axs[1].set_ylabel('Rainfall (mm)')
axs[1].legend()

# กราฟแสดงความชื้น (Humidity)
axs[2].plot(avg_data_filtered['year'].astype(str) + '-' + avg_data_filtered['month'].astype(str), avg_data_filtered['humid_15d_avg'], color='blue', label='Humidity')
axs[2].set_title('Average Humidity (%) from 2017 to 2024')
axs[2].set_ylabel('Humidity (%)')
axs[2].legend()

# ตั้งชื่อกราฟ
plt.xlabel('Date (Year-Month)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# อ่านข้อมูล
df = pd.read_csv('df_final_clean.csv')  # ใส่ path ของไฟล์ที่คุณใช้

# แปลงคอลัมน์ 'date' เป็น datetime
df['date'] = pd.to_datetime(df['date'])

# สร้างคอลัมน์ 'year' และ 'season' สำหรับกราฟรายปีและฤดูกาล
df['year'] = df['date'].dt.year  # เพิ่มคอลัมน์ 'year' สำหรับการกรุ๊ปข้อมูลรายปี
df['season'] = df['date'].dt.month % 12 // 3 + 1  # ฤดูกาล: 1=ฤดูร้อน, 2=ฤดูฝน, 3=ฤดูหนาว

# คำนวณค่าเฉลี่ยประจำปี
df_yearly = df.groupby('year').agg({
    'humid_15d_avg': 'mean',
    'rain_15d_avg': 'mean',
    'temp_15d_avg': 'mean'
}).reset_index()

# สร้างกราฟ
fig, ax = plt.subplots(3, 1, figsize=(10, 12))

# กราฟแสดงอุณหภูมิ
ax[0].plot(df_yearly['year'], df_yearly['temp_15d_avg'], label='Temperature', color='red')
ax[0].set_title('Average Temperature from 2017 to 2024')
ax[0].set_xlabel('Year')
ax[0].set_ylabel('Temperature (°C)')

# กราฟแสดงฝน
ax[1].plot(df_yearly['year'], df_yearly['rain_15d_avg'], label='Rain', color='blue')
ax[1].set_title('Average Rainfall from 2017 to 2024')
ax[1].set_xlabel('Year')
ax[1].set_ylabel('Rainfall (mm)')

# กราฟแสดงความชื้น
ax[2].plot(df_yearly['year'], df_yearly['humid_15d_avg'], label='Humidity', color='green')
ax[2].set_title('Average Humidity from 2017 to 2024')
ax[2].set_xlabel('Year')
ax[2].set_ylabel('Humidity (%)')

# แสดงกราฟ
plt.tight_layout()
plt.legend()
plt.show()


In [ ]:
print(df.columns)


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import font_manager

# กำหนดฟอนต์ที่ใช้
font_path = "/content/THSarabunNew.ttf"  # แก้ไขให้ตรงกับตำแหน่งฟอนต์ที่ติดตั้ง
font_prop = font_manager.FontProperties(fname=font_path)

# คำนวณจำนวนผู้ป่วยทั้งหมดในแต่ละตำบล
top_subdistricts = df_clean.groupby('subdistrict')['cases'].sum().sort_values(ascending=False).head(20)

# สร้างกราฟ
plt.figure(figsize=(12, 8))
top_subdistricts.plot(kind='barh', color='lightblue')

# ตั้งชื่อกราฟและแกน
plt.xlabel('Total Cases', fontproperties=font_prop)
plt.ylabel('Subdistrict', fontproperties=font_prop)
plt.title('Top 20 Subdistricts by Total Cases', fontproperties=font_prop)

# หมุนชื่อให้สามารถอ่านได้ง่ายขึ้น
plt.xticks(rotation=45, fontproperties=font_prop)
plt.yticks(fontproperties=font_prop)

# เพิ่มข้อความแสดงจำนวนผู้ป่วย
for index, value in enumerate(top_subdistricts):
    plt.text(value, index, str(value), fontproperties=font_prop)  # แสดงค่าในแต่ละตำแหน่งของบาร์

plt.show()


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import font_manager

# กำหนดฟอนต์ที่ใช้
font_path = "/content/THSarabunNew.ttf"  # แก้ไขให้ตรงกับตำแหน่งฟอนต์ที่ติดตั้ง
font_prop = font_manager.FontProperties(fname=font_path)

# คำนวณจำนวนผู้ป่วยทั้งหมดในแต่ละตำบล
top_subdistricts = df_clean.groupby('subdistrict')['cases'].sum().sort_values(ascending=True).head(20)  # เลือก 20 ตำบลที่มีจำนวนผู้ป่วยน้อยที่สุด

# สร้างกราฟ
plt.figure(figsize=(12, 8))
top_subdistricts.plot(kind='barh', color='lightblue')

# ตั้งชื่อกราฟและแกน
plt.xlabel('Total Cases', fontproperties=font_prop)
plt.ylabel('Subdistrict', fontproperties=font_prop)
plt.title('Top 20 Subdistricts by Total Cases (Lowest)', fontproperties=font_prop)

# หมุนชื่อให้สามารถอ่านได้ง่ายขึ้น
plt.xticks(rotation=45, fontproperties=font_prop)
plt.yticks(fontproperties=font_prop)

# เพิ่มข้อความแสดงจำนวนผู้ป่วย
for index, value in enumerate(top_subdistricts):
    plt.text(value, index, str(value), fontproperties=font_prop)  # แสดงค่าในแต่ละตำแหน่งของบาร์

plt.show()


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import font_manager

# กำหนดฟอนต์ที่ใช้
font_path = "/content/THSarabunNew.ttf"  # แก้ไขให้ตรงกับตำแหน่งฟอนต์ที่ติดตั้ง
font_prop = font_manager.FontProperties(fname=font_path)

# คำนวณจำนวนผู้ป่วยทั้งหมดในแต่ละอาชีพ
top_occupation = df_clean.groupby('occupation')['cases'].sum().sort_values(ascending=False).head(20)

# สร้างกราฟ
plt.figure(figsize=(12, 10))  # เพิ่มขนาดกราฟ
top_occupation.plot(kind='barh', color='lightcoral')  # ใช้ top_occupation แทน top_subdistricts

# ตั้งชื่อกราฟและแกน
plt.xlabel('Total Cases', fontproperties=font_prop)
plt.ylabel('Occupation', fontproperties=font_prop)  # เปลี่ยนชื่อแกน y
plt.title('Top 20 Occupation by Total Cases', fontproperties=font_prop)  # เปลี่ยนชื่อกราฟ

# หมุนชื่อให้สามารถอ่านได้ง่ายขึ้น
plt.xticks(rotation=45, fontproperties=font_prop)
plt.yticks(fontproperties=font_prop)

# เพิ่มข้อความแสดงจำนวนผู้ป่วย
for index, value in enumerate(top_occupation):  # ใช้ top_occupation แทน top_subdistricts
    plt.text(value, index, str(value), fontproperties=font_prop)  # แสดงค่าในแต่ละตำแหน่งของบาร์

plt.tight_layout()  # จัดการกับขนาดของกราฟให้สมบูรณ์
plt.show()


# ขั้นตอน 8: วิเคราะห์เชิงเดี่ยว (Univariate Analysis)

In [ ]:
# ตรวจสอบชื่อคอลัมน์อีกครั้ง
print(df_final_clean.columns)


In [ ]:
import pandas as pd
import scipy.stats as stats
from scipy.stats import spearmanr, pointbiserialr, kruskal

# เลือกตัวแปรที่ใช้
target = 'cases'
numeric_vars = ['temp_15d_avg', 'rain_15d_avg', 'humid_15d_avg', 'อายุ(ปี)']  # รวม 'อายุ(ปี)'
binary_cats = ['เพศ']
multi_cats = ['อาชีพ', 'ตำบล', 'อำเภอ', 'เดือน', 'ปี', 'ไตรมาส', 'ฤดูกาล']  # รวมตัวแปร 'เดือน', 'ปี', 'ไตรมาส', 'ฤดูกาล'

# แปลงเพศเป็นตัวเลข (ชาย=0, หญิง=1)
if 'เพศ' in df_final_clean.columns:
    df_final_clean['gender_code'] = df_final_clean['เพศ'].map({'ชาย': 0, 'หญิง': 1})

# ลบ missing values ที่จำเป็นสำหรับตัวแปรที่ใช้ในการคำนวณ
df_final_clean = df_final_clean.dropna(subset=['temp_15d_avg', 'rain_15d_avg', 'humid_15d_avg', 'gender_code', target])

# คำนวณค่า p-value
results = []

# 1) Spearman correlation กับตัวแปรเชิงปริมาณ
for var in numeric_vars:
    corr, p = spearmanr(df_final_clean[var], df_final_clean[target])
    results.append({
        'Variable': var,
        'Test': 'Spearman Correlation',
        'Stat': round(corr, 3),
        'P-Value': round(p, 4)
    })

# 2) Point-Biserial correlation กับตัวแปรไบนารี
for var in binary_cats:
    encoded_var = var + '_code'  # ชื่อคอลัมน์ที่แปลงจากเพศ
    if encoded_var in df_final_clean.columns and df_final_clean[encoded_var].nunique() == 2:
        corr, p = pointbiserialr(df_final_clean[encoded_var], df_final_clean[target])
        results.append({
            'Variable': var,
            'Test': 'Point-Biserial',
            'Stat': round(corr, 3),
            'P-Value': round(p, 4)
        })

# 3) Kruskal-Wallis test กับตัวแปรหลายกลุ่ม
for var in multi_cats:
    df_non_null = df_final_clean.dropna(subset=[var])
    groups = [group[target].dropna() for name, group in df_non_null.groupby(var)]
    if len(groups) > 1:
        h_stat, p = kruskal(*groups)
        results.append({
            'Variable': var,
            'Test': 'Kruskal-Wallis',
            'Stat': round(h_stat, 3),
            'P-Value': round(p, 4)
        })

# แสดงผล
results_df = pd.DataFrame(results).sort_values('P-Value')
print("\n📊 ผลวิเคราะห์เชิงเดี่ยว (เรียงตาม p-value):")
print(results_df)


In [ ]:
import pandas as pd
import scipy.stats as stats
from scipy.stats import spearmanr, pointbiserialr, kruskal

# เลือกตัวแปรที่ใช้
target = 'cases'
numeric_vars = ['temp_15d_avg', 'rain_15d_avg', 'humid_15d_avg', 'อายุ(ปี)']  # รวม 'อายุ(ปี)'
binary_cats = ['เพศ']
multi_cats = ['อาชีพ', 'ตำบล', 'อำเภอ', 'เดือน', 'ปี', 'ไตรมาส', 'ฤดูกาล']  # รวมตัวแปร 'เดือน', 'ปี', 'ไตรมาส', 'ฤดูกาล'

# แปลงเพศเป็นตัวเลข (ชาย=0, หญิง=1)
if 'เพศ' in df_final_clean.columns:
    df_final_clean['gender_code'] = df_final_clean['เพศ'].map({'ชาย': 0, 'หญิง': 1})

# ลบ missing values ที่จำเป็นสำหรับตัวแปรที่ใช้ในการคำนวณ
df_final_clean = df_final_clean.dropna(subset=['temp_15d_avg', 'rain_15d_avg', 'humid_15d_avg', 'gender_code', target])

# คำนวณค่า p-value
results = []

# 1) Spearman correlation กับตัวแปรเชิงปริมาณ
for var in numeric_vars:
    corr, p = spearmanr(df_final_clean[var], df_final_clean[target])
    results.append({
        'Variable': var,
        'Test': 'Spearman Correlation',
        'Stat': round(corr, 3),
        'P-Value': round(p, 4)
    })

# 2) Point-Biserial correlation กับตัวแปรไบนารี
for var in binary_cats:
    encoded_var = var + '_code'  # ชื่อคอลัมน์ที่แปลงจากเพศ
    if encoded_var in df_final_clean.columns and df_final_clean[encoded_var].nunique() == 2:
        corr, p = pointbiserialr(df_final_clean[encoded_var], df_final_clean[target])
        results.append({
            'Variable': var,
            'Test': 'Point-Biserial',
            'Stat': round(corr, 3),
            'P-Value': round(p, 4)
        })

# 3) Kruskal-Wallis test กับตัวแปรหลายกลุ่ม
for var in multi_cats:
    df_non_null = df_final_clean.dropna(subset=[var])
    groups = [group[target].dropna() for name, group in df_non_null.groupby(var)]
    if len(groups) > 1:
        h_stat, p = kruskal(*groups)
        results.append({
            'Variable': var,
            'Test': 'Kruskal-Wallis',
            'Stat': round(h_stat, 3),
            'P-Value': round(p, 4)
        })

# แปลงผลลัพธ์เป็น DataFrame
results_df = pd.DataFrame(results).sort_values('P-Value')

# กรองเฉพาะตัวแปรที่มี p-value < 0.05
significant_results_df = results_df[results_df['P-Value'] < 0.05]

# แสดงผล
print("\n📊 ผลวิเคราะห์เชิงเดี่ยว (ตัวแปรที่มี p-value < 0.05):")
print(significant_results_df)


# ขั้นตอน 9: วิเคราะห์เชิงพหุด้วย Negative Binomial Regression

In [ ]:
print(df_final_clean.columns)  # ตรวจสอบคอลัมน์ทั้งหมดใน DataFrame


In [ ]:
df_final_clean.rename(columns={
    'date': 'date',
    'cases': 'cases',
    'temp_15d_avg': 'temp_15d_avg',
    'rain_15d_avg': 'rain_15d_avg',
    'humid_15d_avg': 'humid_15d_avg',
    'อายุ(ปี)': 'age',
    'เพศ': 'gender',
    'อาชีพ': 'occupation',
    'ตำบล': 'subdistrict',
    'อำเภอ': 'district',
    'เดือน': 'month',
    'ปี': 'year',
    'ไตรมาส': 'quarter',
    'ฤดูกาล': 'season',
    'gender_code': 'gender_code'
}, inplace=True)


In [ ]:
import statsmodels.api as sm  # นำเข้า statsmodels


In [ ]:
# แปลงคอลัมน์ที่เป็น categorical ให้เป็น dtype 'category' (ตัวแปรหมวดหมู่)
df_final_clean['occupation'] = df_final_clean['occupation'].astype('category')
df_final_clean['subdistrict'] = df_final_clean['subdistrict'].astype('category')
df_final_clean['district'] = df_final_clean['district'].astype('category')
df_final_clean['season'] = df_final_clean['season'].astype('category')
df_final_clean['quarter'] = df_final_clean['quarter'].astype('category')

# ตรวจสอบข้อมูลใหม่
print(df_final_clean.dtypes)


In [ ]:
# ลบคอลัมน์ season ที่ซ้ำ
df_final_clean = df_final_clean.loc[:, ~df_final_clean.columns.duplicated()]

# สร้างโมเดล Negative Binomial
nb_model = smf.glm(
    formula=formula,
    data=df_final_clean,
    family=sm.families.NegativeBinomial()  # ใช้ sm.families.NegativeBinomial()
).fit()

# แสดงผลลัพธ์ของโมเดล
print(nb_model.summary())


In [ ]:
# สรุปเฉพาะตัวแปรที่มีนัยสำคัญ p-value < 0.05
summary_df = nb_model.summary2().tables[1]
significant_vars = summary_df[summary_df['P>|z|'] < 0.05]

print("\n📌 ตัวแปรที่มีนัยสำคัญ (p-value < 0.05):")
print(significant_vars[['Coef.', 'Std.Err.', 'z', 'P>|z|']])


In [ ]:
# ตรวจสอบ overdispersion
mean_cases = df_final_clean['cases'].mean()
var_cases = df_final_clean['cases'].var()

print(f"\nMean of cases: {mean_cases:.4f}")
print(f"Variance of cases: {var_cases:.4f}")

# ตรวจสอบว่าเกิด overdispersion หรือไม่
if var_cases > mean_cases:
    print("📈 มี overdispersion → เหมาะกับ Negative Binomial")
else:
    print("📉 ไม่มี overdispersion → อาจไม่เหมาะกับ Negative Binomial")


In [ ]:
from sklearn.preprocessing import StandardScaler

# การปรับข้อมูลให้เป็นมาตรฐาน (Standardization)
scaler = StandardScaler()

# ทำการ normalize ข้อมูลที่ใช้ในการพยากรณ์
df_final_clean[['temp_15d_avg', 'rain_15d_avg', 'humid_15d_avg']] = scaler.fit_transform(
    df_final_clean[['temp_15d_avg', 'rain_15d_avg', 'humid_15d_avg']])

# สร้างโมเดล Negative Binomial ใหม่
nb_model = smf.glm(
    formula=formula,
    data=df_final_clean,
    family=sm.families.NegativeBinomial()
).fit()

# ผลลัพธ์ใหม่
predictions = nb_model.predict(df_final_clean)


In [ ]:
# Use rolling average for smoothing
predictions_smooth = predictions.rolling(window=10).mean()
actual_cases_smooth = df_final_clean['cases'].rolling(window=10).mean()

# กราฟใหม่
plt.figure(figsize=(10,6))
plt.plot(predictions_smooth, label='Predicted Cases', color='blue')
plt.plot(actual_cases_smooth, label='Actual Cases', color='red', alpha=0.6)
plt.title("Smoothed Predicted vs Actual Cases")
plt.xlabel("Index")
plt.ylabel("Number of Cases")
plt.legend()
plt.show()


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# แบ่งข้อมูล
X = df_final_clean_encoded
y = df_final_clean['cases']

# แบ่งข้อมูลเป็นชุดฝึกและทดสอบ
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# สร้างโมเดล XGBoost
model = xgb.XGBRegressor(objective='reg:squarederror', eval_metric='rmse')

# ฝึกโมเดล
model.fit(X_train, y_train)

# ทำนายค่าบนชุดทดสอบ
y_pred = model.predict(X_test)

# คำนวณค่า MSE (Mean Squared Error)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)  # คำนวณค่ารากที่สองของ MSE
print(f"RMSE: {rmse}")

# คำนวณค่า R² (Coefficient of Determination)
r2 = r2_score(y_test, y_pred)
print(f"R²: {r2}")

# กราฟเปรียบเทียบผลทำนายและค่าจริง
plt.figure(figsize=(10,6))
plt.plot(y_test.values, label='Actual Cases', color='red')
plt.plot(y_pred, label='Predicted Cases', color='blue')
plt.title('Predicted vs Actual Cases')
plt.xlabel('Index')
plt.ylabel('Number of Cases')
plt.legend()
plt.show()

# แสดง Feature Importance
xgb.plot_importance(model, importance_type='weight', max_num_features=10)
plt.show()


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt

# เลือกเฉพาะตัวแปรที่มีนัยสำคัญ
significant_columns = ['month', 'rain_15d_avg', 'humid_15d_avg', 'subdistrict']

# แปลง categorical variables ให้เป็นตัวเลข
df_final_clean_encoded = df_final_clean[significant_columns].copy()

# ใช้ LabelEncoder สำหรับแปลงตัวแปรเชิงหมวดหมู่ให้เป็นตัวเลข
label_encoder = LabelEncoder()
df_final_clean_encoded['subdistrict'] = label_encoder.fit_transform(df_final_clean_encoded['subdistrict'])

# สร้าง X และ y
X = df_final_clean_encoded
y = df_final_clean['cases']

# แบ่งข้อมูลเป็นชุดฝึกและทดสอบ
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# สร้างโมเดล XGBoost และปรับพารามิเตอร์
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    eval_metric='rmse',
    max_depth=5,        # ลดความลึกของต้นไม้
    learning_rate=0.05,  # ลดค่าความเร็วในการเรียนรู้
    n_estimators=500,   # เพิ่มจำนวนต้นไม้
    colsample_bytree=0.8,  # ตัวแปรที่ใช้ในการแบ่งแต่ละต้นไม้
    subsample=0.7,         # ขนาดของแต่ละชุดย่อยในการฝึก
    gamma=0.1           # การปรับให้โมเดลไม่ฟิตมากเกินไป
)

# ฝึกโมเดล
model.fit(X_train, y_train)

# ทำนายค่าบนชุดทดสอบ
y_pred = model.predict(X_test)

# คำนวณค่า MSE (Mean Squared Error)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)  # คำนวณค่ารากที่สองของ MSE
print(f"RMSE: {rmse}")

# คำนวณค่า R² (Coefficient of Determination)
r2 = r2_score(y_test, y_pred)
print(f"R²: {r2}")

# กราฟเปรียบเทียบผลทำนายและค่าจริง
plt.figure(figsize=(10,6))
plt.plot(y_test.values, label='Actual Cases', color='red')
plt.plot(y_pred, label='Predicted Cases', color='blue')
plt.title('Predicted vs Actual Cases')
plt.xlabel('Index')
plt.ylabel('Number of Cases')
plt.legend()
plt.show()

# คำนวณ Feature Importance
xgb.plot_importance(model, importance_type='weight', max_num_features=10, title="Feature Importance")
plt.show()


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# เลือกเฉพาะตัวแปรที่มีนัยสำคัญ
significant_columns = ['rain_15d_avg', 'humid_15d_avg', 'month', 'subdistrict']

# แปลง categorical variables ให้เป็นตัวเลข
df_final_clean_encoded = df_final_clean[significant_columns].copy()

# ใช้ One-Hot Encoding สำหรับแปลงตัวแปร subdistrict เป็นค่าดิจิตอล
df_final_clean_encoded = pd.get_dummies(df_final_clean_encoded, columns=['subdistrict'], drop_first=True)

# ใช้ LabelEncoder สำหรับแปลงตัวแปรเดือน
label_encoder = LabelEncoder()
df_final_clean_encoded['month'] = label_encoder.fit_transform(df_final_clean_encoded['month'])

# สร้าง X และ y
X = df_final_clean_encoded
y = df_final_clean['cases']

# แบ่งข้อมูลเป็นชุดฝึกและทดสอบ
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# สร้างโมเดล XGBoost และปรับพารามิเตอร์
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    eval_metric='rmse',
    max_depth=5,        # ลดความลึกของต้นไม้
    learning_rate=0.05,  # ลดค่าความเร็วในการเรียนรู้
    n_estimators=500,   # เพิ่มจำนวนต้นไม้
    colsample_bytree=0.8,  # ตัวแปรที่ใช้ในการแบ่งแต่ละต้นไม้
    subsample=0.7,         # ขนาดของแต่ละชุดย่อยในการฝึก
    gamma=0.1           # การปรับให้โมเดลไม่ฟิตมากเกินไป
)

# ฝึกโมเดล
model.fit(X_train, y_train)

# ทำนายค่าบนชุดทดสอบ
y_pred = model.predict(X_test)

# คำนวณค่า MSE (Mean Squared Error)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)  # คำนวณค่ารากที่สองของ MSE
print(f"RMSE: {rmse}")

# คำนวณค่า R² (Coefficient of Determination)
r2 = r2_score(y_test, y_pred)
print(f"R²: {r2}")

# กราฟเปรียบเทียบผลทำนายและค่าจริง
plt.figure(figsize=(10,6))
plt.plot(y_test.values, label='Actual Cases', color='red')
plt.plot(y_pred, label='Predicted Cases', color='blue')
plt.title('Predicted vs Actual Cases')
plt.xlabel('Index')
plt.ylabel('Number of Cases')
plt.legend()
plt.show()

# คำนวณ Feature Importance
xgb.plot_importance(model, importance_type='weight', max_num_features=10, title="Feature Importance")
plt.show()


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

# เลือกเฉพาะตัวแปรที่มีนัยสำคัญ
significant_columns = ['rain_15d_avg', 'humid_15d_avg', 'month', 'subdistrict']

# แปลง categorical variables ให้เป็นตัวเลข
df_final_clean_encoded = df_final_clean[significant_columns].copy()

# ใช้ One-Hot Encoding สำหรับแปลงตัวแปร subdistrict เป็นค่าดิจิตอล
df_final_clean_encoded = pd.get_dummies(df_final_clean_encoded, columns=['subdistrict'], drop_first=True)

# ใช้ LabelEncoder สำหรับแปลงตัวแปรเดือน
label_encoder = LabelEncoder()
df_final_clean_encoded['month'] = label_encoder.fit_transform(df_final_clean_encoded['month'])

# สร้าง X และ y
X = df_final_clean_encoded
y = df_final_clean['cases']

# แบ่งข้อมูลเป็นชุดฝึกและทดสอบ
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#  Scaling สำหรับ Deep Learning
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# XGBoost
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    eval_metric='rmse',
    max_depth=12,  # เพิ่มความลึกของต้นไม้
    learning_rate=0.01,  # ลด learning rate
    n_estimators=2000,  # เพิ่มจำนวนต้นไม้
    colsample_bytree=0.9,  # เลือกคุณสมบัติสำหรับแต่ละต้นไม้
    subsample=0.85,  # ขนาดของการสุ่มข้อมูล
    alpha=0.2,  # เพิ่มการ regularization
    gamma=0.01,  # ความยืดหยุ่นในการป้องกัน overfitting
    random_state=42
)

xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

print("XGBoost")
print("  RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_xgb)))
print("  R2  :", r2_score(y_test, y_pred_xgb))

# Random Forest
# Random Forest
rf_model = RandomForestRegressor(n_estimators=500, max_depth=20, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

print("\nRandom Forest")
print("  RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_rf)))
print("  R2  :", r2_score(y_test, y_pred_rf))

# Deep Learning
dl_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

dl_model.compile(optimizer='adam', loss='mse')

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

dl_model.fit(X_train_scaled, y_train,
             validation_split=0.2,
             epochs=50,  # ลดจำนวน epochs ให้เหมาะสม
             batch_size=16,
             verbose=1,
             callbacks=[early_stop])

y_pred_dl = dl_model.predict(X_test_scaled).flatten()

print("\nDeep Learning")
print("  RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_dl)))
print("  R2  :", r2_score(y_test, y_pred_dl))


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt

# เลือกเฉพาะตัวแปรที่มีนัยสำคัญ
significant_columns = ['temp_15d_avg', 'rain_15d_avg', 'humid_15d_avg', 'gender_code', 'occupation',
                       'subdistrict', 'district', 'season', 'quarter', 'year']

# แปลง categorical variables ให้เป็นตัวเลข
df_final_clean_encoded = df_final_clean[significant_columns].copy()

# ใช้ LabelEncoder สำหรับแปลงตัวแปรเชิงหมวดหมู่ให้เป็นตัวเลข
label_encoder = LabelEncoder()
df_final_clean_encoded['occupation'] = label_encoder.fit_transform(df_final_clean_encoded['occupation'])
df_final_clean_encoded['subdistrict'] = label_encoder.fit_transform(df_final_clean_encoded['subdistrict'])
df_final_clean_encoded['district'] = label_encoder.fit_transform(df_final_clean_encoded['district'])
df_final_clean_encoded['season'] = label_encoder.fit_transform(df_final_clean_encoded['season'])
df_final_clean_encoded['quarter'] = label_encoder.fit_transform(df_final_clean_encoded['quarter'])
df_final_clean_encoded['year'] = label_encoder.fit_transform(df_final_clean_encoded['year'])

# สร้าง X และ y
X = df_final_clean_encoded
y = df_final_clean['cases']

# แบ่งข้อมูลเป็นชุดฝึกและทดสอบ
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# สร้างโมเดล Random Forest
rf_model = RandomForestRegressor(n_estimators=200,  max_depth=10, random_state=42)

# ฝึกโมเดล
rf_model.fit(X_train, y_train)

# ทำนายค่าบนชุดทดสอบ
y_pred = rf_model.predict(X_test)

# คำนวณค่า MSE (Mean Squared Error)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)  # คำนวณค่ารากที่สองของ MSE
print(f"RMSE: {rmse}")

# คำนวณค่า R² (Coefficient of Determination)
r2 = r2_score(y_test, y_pred)
print(f"R²: {r2}")

# กราฟเปรียบเทียบผลทำนายและค่าจริง
plt.figure(figsize=(10,6))
plt.plot(y_test.values, label='Actual Cases', color='red')
plt.plot(y_pred, label='Predicted Cases', color='blue')
plt.title('Predicted vs Actual Cases')
plt.xlabel('Index')
plt.ylabel('Number of Cases')
plt.legend()
plt.show()

# Plot feature importance
importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10,6))
plt.title('Feature Importance')
plt.barh(range(X_train.shape[1]), importances[indices], align='center')
plt.yticks(range(X_train.shape[1]), [X_train.columns[i] for i in indices])
plt.xlabel('Importance score')
plt.show()


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt

# เลือกเฉพาะตัวแปรที่มีนัยสำคัญ
significant_columns = ['rain_15d_avg', 'month', 'subdistrict', 'humid_15d_avg']

# แปลง categorical variables ให้เป็นตัวเลข
df_final_clean_encoded = df_final_clean[significant_columns].copy()

# ใช้ LabelEncoder สำหรับแปลงตัวแปรเชิงหมวดหมู่ให้เป็นตัวเลข
label_encoder = LabelEncoder()
df_final_clean_encoded['subdistrict'] = label_encoder.fit_transform(df_final_clean_encoded['subdistrict'])

# สร้าง X และ y
X = df_final_clean_encoded
y = df_final_clean['cases']

# แบ่งข้อมูลเป็นชุดฝึกและทดสอบ
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# สร้างโมเดล Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# ฝึกโมเดล
rf_model.fit(X_train, y_train)

# ทำนายค่าบนชุดทดสอบ
y_pred = rf_model.predict(X_test)

# คำนวณค่า MSE (Mean Squared Error)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)  # คำนวณค่ารากที่สองของ MSE
print(f"RMSE: {rmse}")

# คำนวณค่า R² (Coefficient of Determination)
r2 = r2_score(y_test, y_pred)
print(f"R²: {r2}")

# กราฟเปรียบเทียบผลทำนายและค่าจริง
plt.figure(figsize=(10,6))
plt.plot(y_test.values, label='Actual Cases', color='red')
plt.plot(y_pred, label='Predicted Cases', color='blue')
plt.title('Predicted vs Actual Cases')
plt.xlabel('Index')
plt.ylabel('Number of Cases')
plt.legend()
plt.show()

# Plot feature importance
importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10,6))
plt.title('Feature Importance')
plt.barh(range(X_train.shape[1]), importances[indices], align='center')
plt.yticks(range(X_train.shape[1]), [X_train.columns[i] for i in indices])
plt.xlabel('Importance score')
plt.show()



In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt

# เลือกเฉพาะตัวแปรที่มีนัยสำคัญ
significant_columns = ['rain_15d_avg', 'month', 'subdistrict', 'humid_15d_avg']

# แปลง categorical variables ให้เป็นตัวเลข
df_final_clean_encoded = df_final_clean[significant_columns].copy()

# ใช้ LabelEncoder สำหรับแปลงตัวแปรเชิงหมวดหมู่ให้เป็นตัวเลข
label_encoder = LabelEncoder()
df_final_clean_encoded['subdistrict'] = label_encoder.fit_transform(df_final_clean_encoded['subdistrict'])

# สร้าง X และ y
X = df_final_clean_encoded
y = df_final_clean['cases']

# แบ่งข้อมูลเป็นชุดฝึกและทดสอบ
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# สร้างโมเดล Gradient Boosting
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5)

# ฝึกโมเดล
gb_model.fit(X_train, y_train)

# ทำนายค่าบนชุดทดสอบ
y_pred = gb_model.predict(X_test)

# คำนวณค่า MSE (Mean Squared Error)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)  # คำนวณค่ารากที่สองของ MSE
print(f"RMSE: {rmse}")

# คำนวณค่า R² (Coefficient of Determination)
r2 = r2_score(y_test, y_pred)
print(f"R²: {r2}")

# กราฟเปรียบเทียบผลทำนายและค่าจริง
plt.figure(figsize=(10,6))
plt.plot(y_test.values, label='Actual Cases', color='red')
plt.plot(y_pred, label='Predicted Cases', color='blue')
plt.title('Predicted vs Actual Cases')
plt.xlabel('Index')
plt.ylabel('Number of Cases')
plt.legend()
plt.show()


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# ✅ เลือกตัวแปรที่ใช้
X = df_clean.drop(columns=['cases', 'date'])  # ลบ target และวันที่ทิ้ง
y = df_clean['cases']

# ✅ One-hot encoding
X_encoded = pd.get_dummies(X, drop_first=True)

# ✅ แบ่ง train/test
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# ✅ Scaling (ถ้าจำเป็น, สามารถใช้ได้เมื่อข้อมูลมีขนาดหรือสเกลต่างกัน)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ข้อมูลพร้อมสำหรับการฝึกโมเดล Machine Learning ต่อไป


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# สร้างโมเดล Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# ฝึกโมเดล
rf.fit(X_train_scaled, y_train)

# ทำนายผล
y_pred_rf = rf.predict(X_test_scaled)

# ประเมินผล
print("📊 ผลการประเมิน Random Forest:")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_rf)):.4f}")
print(f"MAE : {mean_absolute_error(y_test, y_pred_rf):.4f}")
print(f"R²  : {r2_score(y_test, y_pred_rf):.4f}")


นัย
# New Section

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

X = df_clean.drop(columns=['cases', 'date'])
y = df_clean['cases']

X_encoded = pd.get_dummies(X, drop_first=True)

# แบ่งข้อมูล
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)


In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, r2_score
import numpy as np
from scipy.stats import uniform, randint

# โมเดลพื้นฐาน
xgb = XGBRegressor(random_state=42)

# ช่วงค่าพารามิเตอร์ที่สุ่มเลือก
param_dist = {
    'n_estimators': randint(100, 600),
    'learning_rate': uniform(0.01, 0.2),
    'max_depth': randint(3, 10),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0.5, 2)
}

# ใช้ r2_score เป็น metric
r2_scorer = make_scorer(r2_score)

# สุ่มเลือก 50 ชุดพารามิเตอร์จาก param_dist
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=50,
    scoring=r2_scorer,
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# รันค้นหาค่าที่ดีที่สุด
random_search.fit(X_train, y_train)

# แสดงผล
print("Best parameters found: ", random_search.best_params_)

print(f"Best R2 score (CV): {random_search.best_score_:.4f}")

# ทดสอบกับ test set
best_xgb = random_search.best_estimator_
y_pred = best_xgb.predict(X_test)

from sklearn.metrics import mean_squared_error, mean_absolute_error

print(f"Test RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")
print(f"Test MAE : {mean_absolute_error(y_test, y_pred):.4f}")
print(f"Test R2  : {r2_score(y_test, y_pred):.4f}")


In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, r2_score
import numpy as np
from scipy.stats import uniform, randint
from sklearn.metrics import mean_squared_error, mean_absolute_error

# โมเดล XGBoost
xgb = XGBRegressor(random_state=42)

# ช่วงค่าพารามิเตอร์ที่สุ่มเลือก
param_dist = {
    'n_estimators': randint(100, 1000),  # ลดจำนวน n_estimators
    'learning_rate': uniform(0.01, 0.1),
    'max_depth': randint(3, 10),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0.5, 2)
}

# ใช้ r2_score เป็น metric
r2_scorer = make_scorer(r2_score)

# สุ่มเลือก 50 ชุดพารามิเตอร์จาก param_dist
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=50,  # ลดจำนวนการสุ่มพารามิเตอร์
    scoring=r2_scorer,
    cv=5,  # ลด cross-validation เป็น 5-fold
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# รันค้นหาค่าที่ดีที่สุด
random_search.fit(X_train, y_train)

# แสดงผลพารามิเตอร์ที่ดีที่สุด
print("Best parameters found: ", random_search.best_params_)
print(f"Best R² score (CV): {random_search.best_score_:.4f}")

# ทดสอบกับ test set
best_xgb = random_search.best_estimator_
y_pred = best_xgb.predict(X_test)

# ประเมินผล
print(f"Test RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")
print(f"Test MAE: {mean_absolute_error(y_test, y_pred):.4f}")
print(f"Test R²: {r2_score(y_test, y_pred):.4f}")


In [ ]:
print(X_encoded_selected.columns)